Importar librerias


In [ ]:
import pandas as pd #libreria de python para manipular datos
import numpy as np # libreria de python para calculos numericos y analisis
import os #modulo para usar funcionalidades del sistema operativo
import oracledb
import sys
from pathlib import Path # modulo de la ruta que se usa
from heapq import merge # modulo para unir Dataframes
import re
import csv
import datetime
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from ipynb.fs.defs.ConexionBD import ConectarBD
from ipynb.fs.defs.ConexionBD import ConectarP
from ipynb.fs.defs.ConexionBD import ConectarAgendamiento
from ipynb.fs.defs.ConexionBD import ConectarAC
from ipynb.fs.defs.ConexionBD import ConectarM
pd.set_option('display.max_columns',None)#Cuantas columnas mostar por Dataframe
csv.field_size_limit(100000000)

In [ ]:
fecha_actual = datetime.datetime.now().date()
hora_actual = datetime.datetime.now().time()
hora_actual = hora_actual.strftime('%H/%M/%S')
ayer = str(fecha_actual - datetime.timedelta(days=1))

In [ ]:
def lowercase_if_not_numeric(x): #funcion para validar si hay alguna identidad de solo numeros y la retorne sin modificar
    if not str(x).isnumeric() :
        y=x.lower()
        return y
    else:
        return x

In [ ]:
#codigo para establecer las rutas para leer y crear archivos

separador = os.path.sep #busca el separador que sería / o \
current_dir = os.path.dirname(os.path.realpath('CruceUsuariosAPP.ipynb'))  #busca la ruta del archivo en el que está CruceUsuariosAPP.ipynb
dir = separador.join(current_dir.split(separador)[:-1]) #se devuelve una carpeta
filenameT = os.path.join(dir, "Data\\APP") #se ruta para leer los export de los aplicativos
filename = os.path.join(dir, "Data\\USUARIOS") #se ruta para leer los usuarios y el SSFF
fileResults= os.path.join(dir,"Resultado\\Primario") #se ruta para guardar el archivo generado 

Importar usuarios

In [ ]:
########################
#Lee el archivo de los usuarios a cruzar
########################
file_exists_Usuarios = os.path.exists(filename+"\\USUARIOS.xlsx")#Pregunta si en la ruta existe el archivo

print(file_exists_Usuarios) #muestra si encuentra la ruta (false o true)


if file_exists_Usuarios == True: #Si el archivo existe se realizarán 

    Usuarios=pd.read_excel(filename+"\\USUARIOS.xlsx") #importa el archivo csv separado por punto y comas
    Usuarios.columns = Usuarios.columns.str.strip() #Quita espacios de los encabezados
    Usuarios.columns=Usuarios.columns.str.upper() #convierte los encabezados de "usuarios" en Mayuscula
    Usuarios= Usuarios.drop_duplicates() # elimina duplicados
    Usuarios = Usuarios.dropna(subset=["DOCUMENTO"])
    
    #se normaliza la data
    Usuarios['DOCUMENTO'] = Usuarios['DOCUMENTO'].astype('Int64')
    Usuarios['DOCUMENTO'] = Usuarios['DOCUMENTO'].astype(str)
    Usuarios['DOCUMENTO'] = Usuarios['DOCUMENTO'].astype(str).str.replace(".", "", regex=True)
    Usuarios['DOCUMENTO'] = Usuarios['DOCUMENTO'].astype(str).str.replace("?", "", regex=True)
    Usuarios['DOCUMENTO'] = [ele.lstrip(' ') for ele in Usuarios['DOCUMENTO']]
    Usuarios['DOCUMENTO'] = [ele.rstrip(' ') for ele in Usuarios['DOCUMENTO']]
    Usuarios["NOMBRE"]= Usuarios["NOMBRE"].str.lower()
    Usuarios['NOMBRE'] = Usuarios['NOMBRE'].astype(str).str.replace("|", "", regex=True)
    Usuarios['NOMBRE'] = Usuarios['NOMBRE'].astype(str).str.replace(".", "", regex=True)
    Usuarios['NOMBRE'] = Usuarios['NOMBRE'].astype(str).str.replace("  ", " ", regex=True)
    Usuarios['NOMBRE'] = Usuarios['NOMBRE'].astype(str).str.replace("á", "a", regex=True)
    Usuarios['NOMBRE'] = Usuarios['NOMBRE'].astype(str).str.replace("é", "e", regex=True)
    Usuarios['NOMBRE'] = Usuarios['NOMBRE'].astype(str).str.replace("í", "i", regex=True)
    Usuarios['NOMBRE'] = Usuarios['NOMBRE'].astype(str).str.replace("ó", "o", regex=True)
    Usuarios['NOMBRE'] = Usuarios['NOMBRE'].astype(str).str.replace("ú", "u", regex=True)
    Usuarios['NOMBRE'] = Usuarios['NOMBRE'].astype(str).str.replace("ñ", "n", regex=True)
    Usuarios['NOMBRE'] = [ele.lstrip(' ') for ele in Usuarios['NOMBRE']]
    Usuarios['NOMBRE'] = [ele.rstrip(' ') for ele in Usuarios['NOMBRE']]
    
    Usuarios['NOMBRE'] = Usuarios['NOMBRE'].replace('\s+', ' ', regex=True)
    Usuarios = Usuarios[Usuarios['NOMBRE'].str.contains(r'\b \b')]
    Usuarios['NOMBRE'] = Usuarios['NOMBRE'].replace('\t', ' ', regex=True)
    
    Usuarios = Usuarios.dropna(subset=["NOMBRE"])
    Cedulas=Usuarios[['DOCUMENTO']]
    Cedulas=Cedulas.drop_duplicates()
    lista1= Cedulas['DOCUMENTO'].tolist() # convierte las cedulas en una lista, esto para hacer el filtro mas adelante
    documentos_de_busqueda = '|'.join(lista1)
    Nombres=Usuarios[['NOMBRE']]
    lista2=Nombres['NOMBRE'].tolist() # convierte los nombre en una lista, esto para hacer el filtro mas adelante
    name = Usuarios["NOMBRE"].str.split( n=3, expand=True)
    name= pd.DataFrame(name)
    name.columns = ['nombre1', 'nombre2','apellido1', 'apellido2']
    Usuarios = pd.concat([Usuarios, name], axis=1)
    Usuarios = Usuarios.fillna({'nombre1': '1', 'nombre2': '2','apellido1': '3', 'apellido2': '4'})
    
    Usuarios['LLAVE1'] = Usuarios["apellido1"] + " " + Usuarios["apellido2"]+ " " + Usuarios["nombre1"]+ " " + Usuarios["nombre2"] # U B N H
    Usuarios['LLAVE2'] = Usuarios["nombre2"] + " " + Usuarios["apellido1"]+ " " + Usuarios["nombre1"]                              # H U N
    Usuarios['LLAVE3'] = Usuarios["nombre1"] + " " + Usuarios["apellido1"]+ " " + Usuarios["apellido2"]                            # N U B
    Usuarios['LLAVE4'] = Usuarios["apellido1"] + " " + Usuarios["apellido2"]+ " " + Usuarios["nombre1"]                            # U B N

    Usuarios = Usuarios.replace("  ", " ", regex=True)
    lista3=Usuarios['LLAVE1'].tolist() #crea una llave con los nombres y lo convierte en una lista
    lista4=Usuarios['LLAVE2'].tolist()#crea una llave con los nombres y lo convierte en una lista
    lista6=Usuarios['LLAVE3'].tolist()
    lista7=Usuarios['LLAVE4'].tolist()

    Usuarios['FECHA DE RETIRO'] = pd.to_datetime(Usuarios['FECHA DE RETIRO'], format='%d%m%Y', errors='coerce').dt.date
    # Usuarios['FECHA DE RETIRO']  = datetime.strptime(Usuarios['FECHA DE RETIRO'] , "%d-%m-%Y")
    Usuarios
else:
    sys.exit("no existe el archivo Usuarios")  


In [ ]:
length = len(lista1)

print(length)

In [ ]:
#se crean dataframes para poder relacionar la cedula de los usuarios cuando estos cruce por identidad o llaves de nombre 
#(y la cedula sea distinta)

if file_exists_Usuarios == True:
    Nube1=Usuarios[['NOMBRE', 'DOCUMENTO']]
    Nube1.rename(columns={'DOCUMENTO':'Documento'}, inplace=True)

    Nube2=Usuarios[['NOMBRE', 'DOCUMENTO']]
    Nube2.rename(columns={'DOCUMENTO':'Documento1', 'NOMBRE':'Nombre'}, inplace=True)

    Nube3=Usuarios[['LLAVE1', 'DOCUMENTO']]
    Nube3.rename(columns={'DOCUMENTO':'Documento1', 'LLAVE1':'Nombre'}, inplace=True)

    Nube4=Usuarios[['LLAVE2', 'DOCUMENTO']]
    Nube4.rename(columns={'DOCUMENTO':'Documento1', 'LLAVE2':'Nombre'}, inplace=True)

    Nube6=Usuarios[['LLAVE3', 'DOCUMENTO']]
    Nube6.rename(columns={'DOCUMENTO':'Documento1', 'LLAVE3':'Nombre'}, inplace=True)

    Nube7=Usuarios[['LLAVE4', 'DOCUMENTO']]
    Nube7.rename(columns={'DOCUMENTO':'Documento1', 'LLAVE4':'Nombre'}, inplace=True)

In [ ]:
#se crea el dataframe donde se guardarán los resultados de los cruces
Nube = pd.DataFrame()

In [ ]:
Nube4

CA

In [ ]:
#se lee el export, si este existe se normaliza la data

file_exists_CA = os.path.exists(filenameT+"\\Export CA.xlsx")

print(file_exists_CA)
if file_exists_CA == True:

    CA=pd.read_excel(filenameT+"\\Export CA.xlsx", sheet_name=0) #IMPORTAR
    CA=CA[['Documento',  'Identidad',  'Primer Nombre','Segundo Nombre','Primer Apellido','Segundo Apellido', 'Estado', 'Nombres']] #USAR SOLO ESAS TRES COLUMNAS
    CA = CA.dropna(subset=["Documento"])
    CA = CA.dropna(subset=["Nombres"])
    # pd.options.display.float_format = '{:.0f}'.format
    # CA['Documento']= CA['Documento'].astype('Int64')
    CA['Documento']= CA['Documento'].astype('str')
    CA=CA.replace({"Retirado": "Inactivo"})
    CA = CA.fillna({'Primer Nombre': '','Segundo Nombre': '','Primer Apellido':'', 'Segundo Apellido':''})
    CA["Primer Nombre"] =CA["Primer Nombre"].astype(str)
    CA["Segundo Nombre"] =CA["Segundo Nombre"].astype(str)
    CA["Primer Apellido"] =CA["Primer Apellido"].astype(str)
    CA["Segundo Apellido"] =CA["Segundo Apellido"].astype(str)
    CA['Nombre']= CA["Primer Nombre"] + " " + CA["Segundo Nombre"] + " " + CA["Primer Apellido"] + " " + CA["Segundo Apellido"]

    CA = CA.sort_values('Estado') # Ordena de A-Z por Role (estado)
    CA["Nombre"]= CA["Nombre"].str.lower()
    CA["Nombres"] =CA["Nombres"].astype(str)
    CA["Nombres"]= CA["Nombres"].str.lower()
    CA = CA.dropna(subset=["Identidad"])
    
    CA["Identidad"] =CA["Identidad"].astype(str)
    CA['Identidad'] = [ele.lstrip('.') for ele in CA['Identidad']]
    CA['Identidad'] = [ele.rstrip('.') for ele in CA['Identidad']]
    CA['Identidad'] = [ele.lstrip(' ') for ele in CA['Identidad']]
    CA['Identidad'] = [ele.rstrip(' ') for ele in CA['Identidad']]

    CA['Identidad'] = CA['Identidad'].str.lower()
    #CA['Nombre'] = CA['Nombre'].astype(str).str.replace("  ", " ", regex=True)
    CA['Nombre'] = CA['Nombre'].replace('\s+', ' ', regex=True)
    CA['Nombre'] = CA['Nombre'].replace('\t', ' ', regex=True)
    CA['Nombre'] = CA['Nombre'].astype(str).str.replace("|", "", regex=True)
    CA['Nombre'] = CA['Nombre'].astype(str).str.replace(".", "", regex=True)
    CA['Nombre'] = CA['Nombre'].astype(str).str.replace("á", "a", regex=True)
    CA['Nombre'] = CA['Nombre'].astype(str).str.replace("é", "e", regex=True)
    CA['Nombre'] = CA['Nombre'].astype(str).str.replace("í", "i", regex=True)
    CA['Nombre'] = CA['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    CA['Nombre'] = CA['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    CA['Nombre'] = CA['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    CA['Nombre'] = [ele.lstrip(' ') for ele in CA['Nombre']]
    CA['Nombre'] = [ele.rstrip(' ') for ele in CA['Nombre']]


    #---------------------
    CA['Nombres'] = CA['Nombres'].replace('\s+', ' ', regex=True)
    CA['Nombres'] = CA['Nombres'].astype(str).str.replace("|", "", regex=True)
    CA['Nombres'] = CA['Nombres'].astype(str).str.replace(".", "", regex=True)
    CA['Nombres'] = CA['Nombres'].astype(str).str.replace("á", "a", regex=True)
    CA['Nombres'] = CA['Nombres'].astype(str).str.replace("é", "e", regex=True)
    CA['Nombres'] = CA['Nombres'].astype(str).str.replace("í", "i", regex=True)
    CA['Nombres'] = CA['Nombres'].astype(str).str.replace("ó", "o", regex=True)
    CA['Nombres'] = CA['Nombres'].astype(str).str.replace("ú", "u", regex=True)
    CA['Nombres'] = CA['Nombres'].astype(str).str.replace("ñ", "n", regex=True)
    CA['Nombres'] = [ele.lstrip(' ') for ele in CA['Nombres']]
    CA['Nombres'] = [ele.rstrip(' ') for ele in CA['Nombres']]

    #listaNomCA= CA['Nombres'].tolist()
    #---------------------
    #CA= CA.drop(['Nombres'], axis=1)
    CA['Documento'] = CA['Documento'].astype(str).str.replace(".", "", regex=True)
    CA['Documento'] = CA['Documento'].astype(str).str.replace("?", "", regex=True) 
    CA['Documento'] = CA['Documento'].astype(str).str.replace("X", "", regex=True)
    CA['Documento'] = CA['Documento'].astype(str).str.replace("x", "", regex=True)
    CA = CA.dropna(subset=["Documento"])
    CA['Documento'] = [ele.lstrip('0') for ele in CA['Documento']]
    CA['Documento'] = [ele.lstrip(' ') for ele in CA['Documento']]
    CA['Documento'] = [ele.rstrip(' ') for ele in CA['Documento']]
    CA = CA.fillna({ 'Identidad': 'sin identidad'})
    CaI = CA[['Documento',  'Identidad',  'Nombre', 'Estado']]

else:
    sys.exit("no existe el archivo CA")

In [ ]:
CA

In [ ]:
filtro1 = pd.DataFrame()
documentosCA= CA[['Documento']]
documentosCA=documentosCA['Documento'].to_list()

In [ ]:

if file_exists_CA == True:
    ca1= CA[['Documento',  'Identidad',  'Nombre', 'Estado', 'Nombres']]
    i=0
    for n in lista1:
        i=i+1
        print(i)
        if n in documentosCA:
            filtroDocumento=ca1[ca1['Documento'].str.contains(n)]
            filtroDocumento=filtroDocumento.assign(Cruce='Documento')
            filtroDocumento=filtroDocumento.assign(DocumentoRRHH = n)
            filtro1=pd.concat([filtro1, filtroDocumento])

    filtro2 = ca1[ca1.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro5 = ca1[ca1.Nombres.isin(lista2)]#filtra por nombre completo
    filtro5=filtro5.assign(Cruce='Nombre completo inv')
    filtro5 = filtro5.merge(Nube2, how='left', left_on='Nombres', right_on='Nombre')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH', 'Nombre_x':'Nombre'}, inplace=True)
    filtro5=filtro5.drop(['Nombre_y'], axis=1)

    filtro3=ca1[ca1.Nombre.isin(lista3)]
    #filtro3 = ca1[ca1.Nombre.isin(lista3)]#filtra llave1
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro4 = ca1[ca1.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = ca1[ca1.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = ca1[ca1.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    CA = pd.concat([filtro1, filtro2, filtro5, filtro3, filtro4, filtro6, filtro7])
    CA=CA.drop(['Nombres'], axis=1)
    CA= CA.drop_duplicates(CA.columns[~CA.columns.isin(['Cruce'])], keep='first')
    CA.index = range(CA.shape[0])
    CA=CA.assign(Usuario_Aplicativo='CA')
    CA.insert(loc = 5,column = 'Modulo',value = 'N/A')
    Nube = pd.concat([Nube, CA])
    Nube.index = range(Nube.shape[0])
    Nube["Identidad"]= Nube["Identidad"].str.lower()

In [ ]:
filtro4

In [ ]:
filtro5


In [ ]:
CA

In [ ]:
filtro1

In [ ]:

lista5=Nube[['Identidad']]
lista5 = lista5.dropna(how='all')
lista5['Identidad'] = [ele.lstrip('.') for ele in lista5['Identidad']]
lista5['Identidad'] = [ele.rstrip('.') for ele in lista5['Identidad']]
lista5['Identidad'] = [ele.lstrip(' ') for ele in lista5['Identidad']]
lista5['Identidad'] = [ele.rstrip(' ') for ele in lista5['Identidad']]
lista5["Identidad"]= lista5["Identidad"].str.lower()
lista5=lista5['Identidad'].tolist()
if "sin identidad" in lista5:
        lista5.remove("sin identidad")
Nube5=Nube[['Identidad', 'DocumentoRRHH']]
Nube5.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)

OIM

In [ ]:
file_exists_OIM = os.path.exists(filenameT+"\\IdentidadesOIM.xlsx")


print(file_exists_OIM)
if file_exists_OIM == True:

    OIM=pd.read_excel(filenameT+"\\IdentidadesOIM.xlsx") #IMPORTAR
    OIM.drop([0,1], axis=0, inplace=True) #Eliminar index
    OIM=OIM[['Unnamed: 1','Unnamed: 2',   'Unnamed: 3',  'Unnamed: 6']]
    OIM.rename(columns={'Unnamed: 1':'Identidad', 'Unnamed: 6':'Documento', 'Unnamed: 2':'Nombre' , 'Unnamed: 3':'Estado'}, inplace=True)
    OIM.index = range(OIM.shape[0])

    OIM = OIM.dropna(subset=["Documento"]) #ELIMINAR FILAS  CON EL DOCUMENTO VACIO
    OIM = OIM.dropna(subset=["Identidad"]) #ELIMINAR FILAS  CON IDENTITDAD VACIA
    OIM["Identidad"]= OIM["Identidad"].str.lower()
    OIM["Nombre"]= OIM["Nombre"].str.lower()
    OIM=OIM.replace({"Deleted": "Inactivo", "Disabled": "Inactivo", "Active":"Activo"})
    OIM = OIM.sort_values('Estado') # Ordena de A-Z por Role (estado)  
    OIM['Nombre']= OIM['Nombre'].astype(str)
    OIM['Nombre'] = OIM['Nombre'].replace('\s+', ' ', regex=True)
    OIM['Nombre'] = OIM['Nombre'].replace('\t', ' ', regex=True)
    OIM['Nombre'] = OIM['Nombre'].astype(str).str.replace("|", "", regex=True)
    OIM['Nombre'] = OIM['Nombre'].astype(str).str.replace(".", "", regex=True)
    OIM['Nombre'] = OIM['Nombre'].astype(str).str.replace("á", "a", regex=True)
    OIM['Nombre'] = OIM['Nombre'].astype(str).str.replace("é", "e", regex=True)
    OIM['Nombre'] = OIM['Nombre'].astype(str).str.replace("í", "i", regex=True)
    OIM['Nombre'] = OIM['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    OIM['Nombre'] = OIM['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    OIM['Nombre'] = OIM['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    OIM['Nombre'] = [ele.lstrip(' ') for ele in OIM['Nombre']]
    OIM['Nombre'] = [ele.rstrip(' ') for ele in OIM['Nombre']]
    OIM['Documento'] = OIM['Documento'].astype(str).str.replace(".", "", regex=True)
    OIM['Documento'] = OIM['Documento'].astype(str).str.replace("?", "", regex=True) 
    OIM['Documento'] = OIM['Documento'].astype(str).str.replace("X", "", regex=True)
    OIM['Documento'] = OIM['Documento'].astype(str).str.replace("x", "", regex=True)
    OIM['Identidad'] = [ele.lstrip('.') for ele in OIM['Identidad']]
    OIM['Identidad'] = [ele.rstrip('.') for ele in OIM['Identidad']]
    OIM['Identidad'] = [ele.lstrip(' ') for ele in OIM['Identidad']]
    OIM['Identidad'] = [ele.rstrip(' ') for ele in OIM['Identidad']]
    OIM['Nombre'] = [ele.lstrip(' ') for ele in OIM['Nombre']]
    OIM['Nombre'] = [ele.rstrip(' ') for ele in OIM['Nombre']]
    OIM['Documento'] = [ele.lstrip(' ') for ele in OIM['Documento']]
    OIM['Documento'] = [ele.rstrip(' ') for ele in OIM['Documento']]
    OIM=OIM[['Documento','Identidad', 'Nombre', 'Estado']]
else:
    sys.exit("no existe el archivo Usuarios")  

In [ ]:
if file_exists_OIM == True:
    filtro1 = pd.DataFrame()
    documentosOIM= OIM[['Documento']]
    documentosOIM=documentosOIM['Documento'].to_list()
    oim1=OIM[['Documento','Identidad', 'Nombre', 'Estado']]
    i=0
    for n in lista1:
        i=i+1
        print(i)

        filtroDocumento=oim1[oim1['Documento'].str.contains(n)]
        filtroDocumento=filtroDocumento.assign(Cruce='Documento')
        filtroDocumento=filtroDocumento.assign(DocumentoRRHH = n)
        filtro1=pd.concat([filtro1, filtroDocumento])
    
    filtro5 = oim1[oim1.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro2 = oim1[oim1.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro3 = oim1[oim1.Nombre.isin(lista3)]#filtra llave1
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro4 = oim1[oim1.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = oim1[oim1.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = oim1[oim1.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    OIM=pd.concat([filtro1, filtro5])
    OIM=pd.concat([OIM, filtro2])
    OIM=pd.concat([OIM, filtro3])
    OIM=pd.concat([OIM, filtro4])
    OIM=pd.concat([OIM, filtro6])
    OIM=pd.concat([OIM, filtro7])
    OIM= OIM.drop_duplicates(OIM.columns[~OIM.columns.isin(['Cruce'])], keep='first')
    OIM=OIM.assign(Usuario_Aplicativo='OIM')
    OIM.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listOim = OIM['Identidad'].to_list()
    lista5 = lista5 + listOim
    oim_1=OIM[['Identidad', 'DocumentoRRHH']]
    oim_1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,oim_1])
    Nube5.index = range(Nube5.shape[0])
    Nube5= Nube5.drop_duplicates( keep='first')
    OIM.index = range(OIM.shape[0])

    Nube = pd.concat([Nube, OIM])
    Nube.index = range(Nube.shape[0])

In [ ]:
OIM

In [ ]:
Nube

Importar DA ClaroCo

In [ ]:
#se lee el export, si este existe se normaliza la data

file_exists_ClaroCo = os.path.exists(filenameT+"\\ADDS_ClaroCO.csv")#Pregunta si en la ruta existe el archivo

print(file_exists_ClaroCo)#muestra false o true 

if file_exists_ClaroCo == True:#Si el archivo existe

    Claro=pd.read_csv(filenameT+"\\ADDS_ClaroCO.csv", sep=',',encoding = "utf-8-sig" , engine='python')#importa el archivo csv separado por coma
    Claro=Claro[['Employeeid','Role', 'PreW2K', 'Display']] #Usar solo las columnas que necesitamos
    #Claro = Claro.dropna(subset=["Employeeid"]) #Eliminar las filas sin documento
    Claro = Claro.dropna(subset=["Role"]) #Eliminar las filas sin Estado
    Claro=Claro.replace({"Enabled, Password Doesnt Expire": "Activo", "Normal_Account": "Activo", 
    "Enabled, Password Not Required":"Activo", "Enabled; Password Doesnt Expire":"Activo",
    "Disabled, Password Doesnt Expire": "Inactivo", "Disabled Account":"Inactivo",
    "Disabled" : "Inactivo", "Disabled, Password Not Required": "Inactivo"}) # Cambia los diferentes estados a Activo e Inactivo
    Claro = Claro.sort_values('Role') # Ordena de A-Z por Role (estado)
    Claro.rename(columns={'PreW2K':'Identidad', 'Employeeid':'Documento', 'Role':'Estado', 'Display':'Nombre'}, inplace=True)
    Claro["Nombre"]= Claro["Nombre"].str.lower()
    Claro['Nombre'] = Claro['Nombre'].astype(str)
    Claro['Identidad'] = [ele.lstrip('.') for ele in Claro['Identidad']]
    Claro['Identidad'] = [ele.rstrip('.') for ele in Claro['Identidad']]
    Claro['Identidad'] = [ele.lstrip(' ') for ele in Claro['Identidad']]
    Claro['Identidad'] = [ele.rstrip(' ') for ele in Claro['Identidad']]
    Claro['Nombre'] = [ele.lstrip(' ') for ele in Claro['Nombre']]
    Claro['Nombre'] = [ele.rstrip(' ') for ele in Claro['Nombre']]
    Claro["Identidad"]= Claro["Identidad"].str.lower()
    Claro['Nombre'] = Claro['Nombre'].replace('\s+', ' ', regex=True)
    Claro['Nombre'] = Claro['Nombre'].replace('\t', ' ', regex=True)
    Claro['Nombre'] = Claro['Nombre'].astype(str).str.replace("|", "", regex=True)
    Claro['Nombre'] = Claro['Nombre'].astype(str).str.replace(".", "", regex=True)
    Claro['Nombre'] = Claro['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Claro['Nombre'] = Claro['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Claro['Nombre'] = Claro['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Claro['Nombre'] = Claro['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Claro['Nombre'] = Claro['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Claro['Nombre'] = Claro['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Claro['Documento'] = Claro['Documento'].astype(str).str.replace(".", "", regex=True)
    Claro['Documento'] = Claro['Documento'].astype(str).str.replace("?", "", regex=True)
    Claro['Documento'] = Claro['Documento'].astype(str).str.replace("X", "", regex=True)
    Claro['Documento'] = Claro['Documento'].astype(str).str.replace("x", "", regex=True)
    Claro['Documento'] = [ele.lstrip(' ') for ele in Claro['Documento']]
    Claro['Documento'] = [ele.rstrip(' ') for ele in Claro['Documento']]
    Claro = Claro.fillna({ 'Identidad': 'sin identidad'})
else:
    sys.exit("no existe el archivo ClaroCo")  

Filtro ClaroCo

In [ ]:
if file_exists_ClaroCo == True:    
    Claro1=Claro[['Documento', 'Identidad','Nombre', 'Estado']]
    #filtro1=Claro1[Claro1['Documento'].str.contains(documentos_de_busqueda)]
    filtro1 = Claro1[Claro1.Documento.isin(lista1)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro1=filtro1.assign(Cruce='Documento')
    filtro1=filtro1.assign(DocumentoRRHH=filtro1['Documento'])
    filtro1 = filtro1.merge(Nube1, how='left', left_on='Documento', right_on='Documento')
    filtro5 = Claro1[Claro1.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro2 = Claro1[Claro1.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Claro1[Claro1.Nombre.isin(lista3)]#filtra llave1
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Claro1[Claro1.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Claro1[Claro1.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Claro1[Claro1.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    Claro = pd.concat([filtro1, filtro5])
    Claro = pd.concat([Claro, filtro2])
    Claro = pd.concat([Claro, filtro3])
    Claro = pd.concat([Claro, filtro4])
    Claro = pd.concat([Claro, filtro6])
    Claro = pd.concat([Claro, filtro7])

    Claro.index = range(Claro.shape[0])
    Claro = Claro.drop('NOMBRE', axis=1)
    Claro= Claro.drop_duplicates(Claro.columns[~Claro.columns.isin(['Cruce'])], keep='first')
    Claro=Claro.assign(Usuario_Aplicativo='DA CLAROCO') # se agrega una columna para saber que ese filtro fue de ClaroCo
    Claro.insert(loc = 5,column = 'Modulo',value = 'N/A')
    listClaro = Claro['Identidad'].to_list()
    if "sin identidad" in listClaro:
        listClaro.remove("sin identidad") 
    lista5 = lista5 + listClaro
    Cla1=Claro[['Identidad', 'DocumentoRRHH']]
    Cla1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Cla1])
    Nube5.index = range(Nube5.shape[0])
    Nube5= Nube5.drop_duplicates( keep='first')
    Claro.index = range(Claro.shape[0])
    Nube = pd.concat([Nube, Claro])
    Nube.index = range(Nube.shape[0])

In [ ]:
Claro

Importar DA Comcel

In [ ]:
file_exists_Comcel = os.path.exists(filenameT+"\\ADDS_Comcel.csv")

print(file_exists_Comcel)

if file_exists_Comcel == True:

    COMCEL=pd.read_csv(filenameT+"\\ADDS_Comcel.csv", sep=',',encoding = "ISO-8859-1" , engine='python')#importa el archivo csv separado por coma
    COMCEL=COMCEL[['Employeeid','Role', 'PreW2K', 'Display']] #Usar solo las columnas que necesitamos
    #COMCEL = COMCEL.dropna(subset=["Employeeid"]) #Eliminar las filas sin documento
    COMCEL = COMCEL.dropna(subset=["Role"]) #Eliminar las filas sin Estado
    COMCEL=COMCEL.replace({"Enabled, Password Doesnt Expire": "Activo", "Normal_Account": "Activo", 
    "Enabled, Password Not Required":"Activo", "Enabled; Password Doesnt Expire":"Activo",
    "Disabled, Password Doesnt Expire": "Inactivo", "Disabled Account":"Inactivo",
    "Disabled" : "Inactivo", "Disabled, Password Not Required": "Inactivo"}) # Cambia los diferentes estados a Activo e Inactivo
    COMCEL = COMCEL.sort_values('Role') # Ordena de A-Z por Role (estado)
    COMCEL.rename(columns={'PreW2K':'Identidad', 'Employeeid':'Documento', 'Role':'Estado', 'Display':'Nombre'}, inplace=True)
    COMCEL["Nombre"]= COMCEL["Nombre"].str.lower()
    COMCEL["Identidad"]= COMCEL["Identidad"].str.lower()
    COMCEL['Identidad'] = [ele.lstrip('.') for ele in COMCEL['Identidad']]
    COMCEL['Identidad'] = [ele.rstrip('.') for ele in COMCEL['Identidad']]
    COMCEL['Identidad'] = [ele.lstrip(' ') for ele in COMCEL['Identidad']]
    COMCEL['Identidad'] = [ele.rstrip(' ') for ele in COMCEL['Identidad']]
    COMCEL['Nombre'] = COMCEL['Nombre'].replace('\s+', ' ', regex=True)
    COMCEL['Nombre'] = COMCEL['Nombre'].replace('\t', ' ', regex=True)
    COMCEL['Nombre'] = COMCEL['Nombre'].astype(str).str.replace("|", "", regex=True)
    COMCEL['Nombre'] = COMCEL['Nombre'].astype(str).str.replace(".", "", regex=True)
    COMCEL['Nombre'] = COMCEL['Nombre'].astype(str).str.replace("á", "a", regex=True)
    COMCEL['Nombre'] = COMCEL['Nombre'].astype(str).str.replace("é", "e", regex=True)
    COMCEL['Nombre'] = COMCEL['Nombre'].astype(str).str.replace("í", "i", regex=True)
    COMCEL['Nombre'] = COMCEL['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    COMCEL['Nombre'] = COMCEL['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    COMCEL['Nombre'] = COMCEL['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    COMCEL['Documento'] = COMCEL['Documento'].astype(str).str.replace(".", "", regex=True)
    COMCEL['Documento'] = COMCEL['Documento'].astype(str).str.replace("?", "", regex=True)
    COMCEL['Documento'] = COMCEL['Documento'].astype(str).str.replace("X", "", regex=True)
    COMCEL['Documento'] = COMCEL['Documento'].astype(str).str.replace("x", "", regex=True)
    COMCEL['Documento'] = [ele.lstrip(' ') for ele in COMCEL['Documento']]
    COMCEL['Documento'] = [ele.rstrip(' ') for ele in COMCEL['Documento']]
    COMCEL['Nombre'] = [ele.lstrip(' ') for ele in COMCEL['Nombre']]
    COMCEL['Nombre'] = [ele.rstrip(' ') for ele in COMCEL['Nombre']]
    COMCEL = COMCEL.fillna({ 'Identidad': 'sin identidad'})
else:
    sys.exit("no existe el archivo Comcel")  

Filtro COMCEL

In [ ]:
if file_exists_Comcel == True:
    
    Comcel1=COMCEL[['Documento', 'Identidad','Nombre', 'Estado']]
    #filtro1=Comcel1[Comcel1['Documento'].str.contains(documentos_de_busqueda)]
    filtro1 = Comcel1[Comcel1.Documento.isin(lista1)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro1=filtro1.assign(Cruce='Documento')
    filtro1=filtro1.assign(DocumentoRRHH=filtro1['Documento'])
    filtro1 = filtro1.merge(Nube1, how='left', left_on='Documento', right_on='Documento')
    filtro5 = Comcel1[Comcel1.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro2 = Comcel1[Comcel1.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro3 = Comcel1[Comcel1.Nombre.isin(lista3)]#filtra llave1
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    
    filtro4 = Comcel1[Comcel1.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Comcel1[Comcel1.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Comcel1[Comcel1.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    COMCEL = pd.concat([filtro1,filtro5])
    COMCEL = pd.concat([COMCEL,filtro2])
    COMCEL = pd.concat([COMCEL,filtro3])
    COMCEL = pd.concat([COMCEL,filtro4])
    COMCEL = pd.concat([COMCEL,filtro6])
    COMCEL = pd.concat([COMCEL,filtro7])
    COMCEL = COMCEL.drop('NOMBRE', axis=1)
    COMCEL= COMCEL.drop_duplicates(COMCEL.columns[~COMCEL.columns.isin(['Cruce'])], keep='first')
    COMCEL.index = range(COMCEL.shape[0])
    COMCEL=COMCEL.assign(Usuario_Aplicativo='DA COMCEL')
    COMCEL.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listCOMCEL = COMCEL['Identidad'].to_list()
    if "sin identidad" in listCOMCEL:
        listCOMCEL.remove("sin identidad") 
    lista5 = lista5 + listCOMCEL
    Com1=COMCEL[['Identidad', 'DocumentoRRHH']]
    Com1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Com1])
    Nube5.index = range(Nube5.shape[0])
    Nube5= Nube5.drop_duplicates( keep='first')
    COMCEL.index = range(COMCEL.shape[0])

    Nube = pd.concat([Nube, COMCEL])
    Nube.index = range(Nube.shape[0])

In [ ]:
COMCEL

Importar DA coAttla

In [ ]:
file_exists_CoAttla = os.path.exists(filenameT+"\\ADDS_Fija.csv")

print(file_exists_CoAttla)
if file_exists_CoAttla == True:

    COATTLA=pd.read_csv(filenameT+"\\ADDS_Fija.csv", sep=',',encoding = "ISO-8859-1" , engine='python')#importa el archivo csv separado por coma
    COATTLA=COATTLA[['Employeeid','Role', 'PreW2K', 'Display']] #Usar solo las columnas que necesitamos
    #COATTLA = COATTLA.dropna(subset=["Employeeid"]) #Eliminar las filas sin documento
    COATTLA = COATTLA.dropna(subset=["Role"]) #Eliminar las filas sin Estado
    COATTLA=COATTLA.replace({"Enabled, Password Doesnt Expire": "Activo", "Normal_Account": "Activo", 
    "Enabled, Password Not Required":"Activo", "Enabled; Password Doesnt Expire":"Activo",
    "Disabled, Password Doesnt Expire": "Inactivo", "Disabled Account":"Inactivo",
    "Disabled" : "Inactivo", "Disabled, Password Not Required": "Inactivo"}) # Cambia los diferentes estados a Activo e Inactivo
    COATTLA = COATTLA.sort_values('Role') # Ordena de A-Z por Role (estado)
    COATTLA.rename(columns={'PreW2K':'Identidad', 'Employeeid':'Documento', 'Role':'Estado', 'Display':'Nombre'}, inplace=True)
    COATTLA["Nombre"]= COATTLA["Nombre"].str.lower()
    COATTLA["Identidad"]= COATTLA["Identidad"].str.lower()
    COATTLA['Identidad'] = [ele.lstrip('.') for ele in COATTLA['Identidad']]
    COATTLA['Identidad'] = [ele.rstrip('.') for ele in COATTLA['Identidad']]
    COATTLA['Identidad'] = [ele.lstrip(' ') for ele in COATTLA['Identidad']]
    COATTLA['Identidad'] = [ele.rstrip(' ') for ele in COATTLA['Identidad']]
    COATTLA['Nombre'] = COATTLA['Nombre'].replace('\s+', ' ', regex=True)
    COATTLA['Nombre'] = COATTLA['Nombre'].replace('\t', ' ', regex=True)
    COATTLA['Nombre'] = COATTLA['Nombre'].astype(str).str.replace("|", "", regex=True)
    COATTLA['Nombre'] = COATTLA['Nombre'].astype(str).str.replace(".", "", regex=True)
    COATTLA['Nombre'] = COATTLA['Nombre'].astype(str).str.replace("á", "a", regex=True)
    COATTLA['Nombre'] = COATTLA['Nombre'].astype(str).str.replace("é", "e", regex=True)
    COATTLA['Nombre'] = COATTLA['Nombre'].astype(str).str.replace("í", "i", regex=True)
    COATTLA['Nombre'] = COATTLA['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    COATTLA['Nombre'] = COATTLA['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    COATTLA['Nombre'] = COATTLA['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    COATTLA['Nombre'] = [ele.lstrip(' ') for ele in COATTLA['Nombre']]
    COATTLA['Nombre'] = [ele.rstrip(' ') for ele in COATTLA['Nombre']]
    COATTLA['Documento'] = COATTLA['Documento'].astype(str).str.replace(".", "", regex=True)
    COATTLA['Documento'] = COATTLA['Documento'].astype(str).str.replace("?", "", regex=True)
    COATTLA['Documento'] = COATTLA['Documento'].astype(str).str.replace("X", "", regex=True)
    COATTLA['Documento'] = COATTLA['Documento'].astype(str).str.replace("x", "", regex=True)
    COATTLA['Documento'] = [ele.lstrip(' ') for ele in COATTLA['Documento']]
    COATTLA['Documento'] = [ele.rstrip(' ') for ele in COATTLA['Documento']]
    COATTLA = COATTLA.fillna({ 'Identidad': 'sin identidad'})
else:
    sys.exit("no existe el archivo CoAttla") 

In [ ]:
if file_exists_CoAttla == True:
    Coattla1=COATTLA[['Documento', 'Identidad','Nombre', 'Estado']]
    #filtro1=Coattla1[Coattla1['Documento'].str.contains(documentos_de_busqueda)]
    filtro1 = Coattla1[Coattla1.Documento.isin(lista1)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro1=filtro1.assign(Cruce='Documento')
    filtro1=filtro1.assign(DocumentoRRHH=filtro1['Documento'])
    filtro1 = filtro1.merge(Nube1, how='left', left_on='Documento', right_on='Documento')
    filtro5 = Coattla1[Coattla1.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro2 = Coattla1[Coattla1.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Coattla1[Coattla1.Nombre.isin(lista3)]#filtra llave1
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Coattla1[Coattla1.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Coattla1[Coattla1.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Coattla1[Coattla1.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    COATTLA = pd.concat([filtro1, filtro5])
    COATTLA = pd.concat([COATTLA, filtro2])
    COATTLA = pd.concat([COATTLA, filtro3])
    COATTLA = pd.concat([COATTLA, filtro4])
    COATTLA = pd.concat([COATTLA, filtro6])
    COATTLA = pd.concat([COATTLA, filtro7])

    
    COATTLA = COATTLA.drop('NOMBRE', axis=1)
    COATTLA= COATTLA.drop_duplicates(COATTLA.columns[~COATTLA.columns.isin(['Cruce'])], keep='first')
    COATTLA.index = range(COATTLA.shape[0])
    COATTLA=COATTLA.assign(Usuario_Aplicativo='DA COATTLA')
    COATTLA.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listCoAttla = COATTLA['Identidad'].to_list()
    if "sin identidad" in listCoAttla:
        listCoAttla.remove("sin identidad") 
    lista5 = lista5 + listCoAttla
    Coa1=COATTLA[['Identidad', 'DocumentoRRHH']]
    Coa1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Coa1])
    Nube5.index = range(Nube5.shape[0])
    Nube5= Nube5.drop_duplicates( keep='first')
    COATTLA.index = range(COATTLA.shape[0])

    Nube = pd.concat([Nube, COATTLA])
    Nube.index = range(Nube.shape[0])

In [ ]:
COATTLA

In [ ]:
Nube

POLIEDRO

In [ ]:
Poliedro = pd.DataFrame()
poliedrobd=ConectarP()

In [ ]:
poliedrobd

In [ ]:

if not poliedrobd.empty == True:
    Poliedro=poliedrobd[['CEDULA','USUARIO_RED','RESPONSABLE', 'HABILITADO','CODUSUARIO']] #USAR SOLO ESAS TRES COLUMNAS
    Poliedro.rename(columns={'CEDULA':'Documento','USUARIO_RED':'Identidad','RESPONSABLE':'Nombre', 'HABILITADO':'Estado'}, inplace=True)
    Poliedro['Nombre'] =Poliedro['Nombre'].astype(str).str.replace(".", "", regex=True)
    Poliedro["Nombre"]= Poliedro["Nombre"].str.lower()
    Poliedro['Identidad'] =Poliedro['Identidad'].astype(str)
    Poliedro["Identidad"]= Poliedro["Identidad"].str.lower()
    Poliedro['Identidad'] = [ele.lstrip('.') for ele in Poliedro['Identidad']]
    Poliedro['Identidad'] = [ele.rstrip('.') for ele in Poliedro['Identidad']]
    Poliedro['Identidad'] = [ele.lstrip(' ') for ele in Poliedro['Identidad']]
    Poliedro['Identidad'] = [ele.rstrip(' ') for ele in Poliedro['Identidad']]
    Poliedro['CODUSUARIO'] =Poliedro['CODUSUARIO'].astype(str)
    Poliedro["CODUSUARIO"]= Poliedro["CODUSUARIO"].str.lower()
    Poliedro['CODUSUARIO'] = [ele.lstrip('.') for ele in Poliedro['CODUSUARIO']]
    Poliedro['CODUSUARIO'] = [ele.rstrip('.') for ele in Poliedro['CODUSUARIO']]
    Poliedro['CODUSUARIO'] = [ele.lstrip(' ') for ele in Poliedro['CODUSUARIO']]
    Poliedro['CODUSUARIO'] = [ele.rstrip(' ') for ele in Poliedro['CODUSUARIO']]
    Poliedro['Nombre'] = Poliedro['Nombre'].replace('\s+', ' ', regex=True)
    Poliedro['Nombre'] = Poliedro['Nombre'].replace('\t', ' ', regex=True)
    Poliedro['Nombre'] = Poliedro['Nombre'].astype(str).str.replace("|", "", regex=True)
    Poliedro['Nombre'] = Poliedro['Nombre'].astype(str).str.replace(".", "", regex=True)
    Poliedro['Nombre'] = Poliedro['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Poliedro['Nombre'] = Poliedro['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Poliedro['Nombre'] = Poliedro['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Poliedro['Nombre'] = Poliedro['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Poliedro['Nombre'] = Poliedro['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Poliedro['Nombre'] = Poliedro['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Poliedro['Estado'] = Poliedro['Estado'].astype(str).str.replace("1", "Activo", regex=True)
    Poliedro['Estado'] = Poliedro['Estado'].astype(str).str.replace("0", "Inactivo", regex=True)
    Poliedro = Poliedro.fillna({'Documento': '0'})
    Poliedro['Documento'] = Poliedro['Documento'].astype(str).str.replace(".", "", regex=True)
    Poliedro['Documento'] = Poliedro['Documento'].astype(str).str.replace("?", "", regex=True) 
    Poliedro['Documento'] = Poliedro['Documento'].astype(str).str.replace("X", "", regex=True)
    Poliedro['Documento'] = Poliedro['Documento'].astype(str).str.replace("x", "", regex=True)
    Poliedro['Documento'] = [ele.lstrip(' ') for ele in Poliedro['Documento']]
    Poliedro['Documento'] = [ele.rstrip(' ') for ele in Poliedro['Documento']]
    Poliedro['Nombre'] = [ele.lstrip(' ') for ele in Poliedro['Nombre']]
    Poliedro['Nombre'] = [ele.rstrip(' ') for ele in Poliedro['Nombre']]
    Poliedro = Poliedro.fillna({ 'Identidad': 'sin identidad'})


In [ ]:
if not poliedrobd.empty == True:
    #filtro1=Poliedro[Poliedro['Documento'].str.contains(documentos_de_busqueda)]
    filtro1 = Poliedro[Poliedro.Documento.isin(lista1)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro1=filtro1.assign(Cruce='Documento')
    filtro1=filtro1.assign(DocumentoRRHH=filtro1['Documento'])
    filtro5 = Poliedro[Poliedro.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro2 = Poliedro[Poliedro.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Poliedro[Poliedro.Nombre.isin(lista3)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Poliedro[Poliedro.Nombre.isin(lista4)] #filtra llave5
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Poliedro[Poliedro.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Poliedro[Poliedro.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    Poliedro = pd.concat([filtro1, filtro5])
    Poliedro = pd.concat([Poliedro, filtro2])
    Poliedro = pd.concat([Poliedro, filtro3])
    Poliedro = pd.concat([Poliedro, filtro4])
    Poliedro = pd.concat([Poliedro, filtro6])
    Poliedro = pd.concat([Poliedro, filtro7])

    listPoliedro = Poliedro['Identidad'].to_list()
    if "sin identidad" in listPoliedro:
        listPoliedro.remove("sin identidad") 
    lista5 = lista5 + listPoliedro
    Pol1=Poliedro[['Identidad', 'DocumentoRRHH']]
    Pol1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Pol1])

    
    Poliedro=Poliedro[['Documento','CODUSUARIO','Nombre', 'Estado', 'Cruce','DocumentoRRHH']] #USAR SOLO ESAS TRES COLUMNAS
    Poliedro= Poliedro.drop_duplicates(Poliedro.columns[~Poliedro.columns.isin(['Cruce'])], keep='first')
    Poliedro.index = range(Poliedro.shape[0])
    Poliedro.rename(columns={'CODUSUARIO':'Identidad'}, inplace=True)
    Poliedro=Poliedro.assign(Usuario_Aplicativo='POLIEDRO')
    Poliedro.insert(loc = 5,column = 'Modulo',value = 'N/A')
    listPoliedro = Poliedro['Identidad'].to_list()

    if "sin identidad" in listPoliedro:
        listPoliedro.remove("sin identidad") 
    lista5 = lista5 + listPoliedro

    Pol1=Poliedro[['Identidad', 'DocumentoRRHH']]
    Pol1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Pol1])
    Nube5.index = range(Nube5.shape[0])
    Nube5= Nube5.drop_duplicates( keep='first')
if Poliedro.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['POLIEDRO'],
    'Modulo':['N/A']}
    Poliedro = pd.DataFrame(datos)

Nube = pd.concat([Nube, Poliedro])
Nube.index = range(Nube.shape[0])

In [ ]:
Poliedro

RR

In [ ]:
file_exists_RR = os.path.exists(filenameT+"\\RR.csv")

print(file_exists_RR)
RR = pd.DataFrame()
if file_exists_RR == True:

    RR=pd.read_csv(filenameT+"\\RR.csv", sep=';',encoding = "utf-8" , engine='python') #IMPORTAR
    RR = RR.fillna({'AUNOM1': '0', 'AUNOM2': '0','AUAPE1': '0','AUAPE2':'0'})
    RR.columns = RR.columns.str.strip()
    RR['AUNOM1'] = [ele.rstrip(' ') for ele in RR['AUNOM1']]
    RR['AUNOM2'] = [ele.rstrip(' ') for ele in RR['AUNOM2']]
    RR['AUAPE1'] = [ele.rstrip(' ') for ele in RR['AUAPE1']]
    RR['AUAPE2'] = [ele.rstrip(' ') for ele in RR['AUAPE2']]
    RR['AUNOM1'] = [ele.lstrip(' ') for ele in RR['AUNOM1']]
    RR['AUNOM2'] = [ele.lstrip(' ') for ele in RR['AUNOM2']]
    RR['AUAPE1'] = [ele.lstrip(' ') for ele in RR['AUAPE1']]
    RR['AUAPE2'] = [ele.lstrip(' ') for ele in RR['AUAPE2']]
    RR['Nombre'] = RR["AUNOM1"] + " " + RR["AUNOM2"]+ " " + RR["AUAPE1"]+ " " + RR["AUAPE2"]
    RR['Nombre'] =RR['Nombre'].astype(str).str.replace("0", "", regex=True)
    RR['Nombre'] =RR['Nombre'].astype(str).str.replace("  ", " ", regex=True)
    RR=RR[['AUSNAM','AUSUSR','Nombre','AUESTA']] #USAR SOLO ESAS TRES COLUMNAS
    RR.rename(columns={'AUSNAM':'Documento','AUSUSR':'Identidad', 'AUESTA':'Estado'}, inplace=True)
    RR = RR.sort_values('Estado')
    RR['Nombre'] =RR['Nombre'].astype(str).str.replace(".", "", regex=True)
    RR["Nombre"]= RR["Nombre"].str.lower()
    RR.columns = RR.columns.str.strip()
    RR["Identidad"]= RR["Identidad"].str.lower()
    RR['Identidad'] = [ele.lstrip('.') for ele in RR['Identidad']]
    RR['Identidad'] = [ele.rstrip('.') for ele in RR['Identidad']]
    RR['Identidad'] = [ele.lstrip(' ') for ele in RR['Identidad']]
    RR['Identidad'] = [ele.rstrip(' ') for ele in RR['Identidad']]
    RR['Nombre'] = RR['Nombre'].replace('\s+', ' ', regex=True)
    RR['Nombre'] = RR['Nombre'].replace('\t', ' ', regex=True)
    RR['Nombre'] = RR['Nombre'].astype(str).str.replace("|", "", regex=True)
    RR['Nombre'] = RR['Nombre'].astype(str).str.replace(".", "", regex=True)
    RR['Nombre'] = RR['Nombre'].astype(str).str.replace("á", "a", regex=True)
    RR['Nombre'] = RR['Nombre'].astype(str).str.replace("é", "e", regex=True)
    RR['Nombre'] = RR['Nombre'].astype(str).str.replace("í", "i", regex=True)
    RR['Nombre'] = RR['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    RR['Nombre'] = RR['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    RR['Nombre'] = RR['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    RR['Estado'] = RR['Estado'].astype(str).str.replace("A", "Activo", regex=True)
    RR['Estado'] = RR['Estado'].astype(str).str.replace("D", "Activo", regex=True)
    RR['Estado'] = RR['Estado'].astype(str).str.replace("R", "Inactivo", regex=True)
    RR['Identidad'] = RR['Identidad'].astype(str)
    RR['Documento'] = RR['Documento'].astype(str).str.replace(".", "", regex=True)
    RR['Documento'] = RR['Documento'].astype(str).str.replace("?", "", regex=True) 
    RR['Documento'] = RR['Documento'].astype(str).str.replace("X", "", regex=True)
    RR['Documento'] = RR['Documento'].astype(str).str.replace("x", "", regex=True)

    RR['Documento'] = [ele.lstrip('0') for ele in RR['Documento']]
    RR['Documento'] = [ele.lstrip(' ') for ele in RR['Documento']]
    RR['Documento'] = [ele.rstrip(' ') for ele in RR['Documento']]
    RR['Identidad'] = [ele.rstrip(' ') for ele in RR['Identidad']]
    RR['Nombre'] = [ele.lstrip(' ') for ele in RR['Nombre']]
    RR['Nombre'] = [ele.rstrip(' ') for ele in RR['Nombre']]
    RR= RR.drop_duplicates(RR.columns[~RR.columns.isin(['Estado', 'Nombre'])], keep='first')
    RR = RR.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
RR

In [ ]:
if file_exists_RR == True:
    #filtro1=RR[RR['Documento'].str.contains(documentos_de_busqueda)]
    filtro1 = RR[RR.Documento.isin(lista1)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro1=filtro1.assign(Cruce='Documento')
    filtro1=filtro1.assign(DocumentoRRHH=filtro1['Documento'])
    filtro2 = RR[RR.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = RR[RR.Nombre.isin(lista3)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = RR[RR.Nombre.isin(lista4)] #filtra llave5
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = RR[RR.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = RR[RR.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    RR = pd.concat([filtro1, filtro2])
    RR = pd.concat([RR, filtro3])
    RR = pd.concat([RR, filtro4])
    RR = pd.concat([RR, filtro6])
    RR = pd.concat([RR, filtro7])

    RR=RR[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    RR= RR.drop_duplicates(RR.columns[~RR.columns.isin(['Cruce'])], keep='first')
    RR.index = range(RR.shape[0])
    RR=RR.assign(Usuario_Aplicativo='RR')
    RR.insert(loc = 5,column = 'Modulo',value = 'N/A')
    listRR = RR['Identidad'].to_list()
    if "sin identidad" in listRR:
        listRR.remove("sin identidad") 
    lista5 = lista5 + listRR
    RR1=RR[['Identidad', 'DocumentoRRHH']]
    RR1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,RR1])
    Nube5= Nube5.drop_duplicates( keep='first')
if RR.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':[0],
    'Usuario_Aplicativo':['RR'],
    'Modulo':['N/A']}
    RR = pd.DataFrame(datos)
    
Nube = pd.concat([Nube, RR])
Nube.index = range(Nube.shape[0])

In [ ]:
Agendamiento = pd.DataFrame()
Agendamientobd=ConectarAgendamiento()

Agendamiento

In [ ]:
file_exists_AGENDAMIENTO = os.path.exists(filenameT+"\\AGENDAMIENTO.csv")

print(file_exists_AGENDAMIENTO)
# Agendamiento = pd.DataFrame()
if (not Agendamientobd.empty == True) or (file_exists_AGENDAMIENTO ==True):
    if Agendamientobd.empty == True:
        Agendamiento=pd.read_csv(filenameT+"\\AGENDAMIENTO.csv", sep=';',encoding = "utf-8-sig" , engine='python') #IMPORTAR
        Agendamiento=Agendamiento[['IDENTIFICACION','NOMBRE', 'USUARIO', 'ESTADO', 'FECHA_CAMBIO']]
    else:
        Agendamiento=Agendamientobd[['IDENTIFICACION','NOMBRE', 'USUARIO', 'ESTADO', 'FECHA_CAMBIO']]
        
    Agendamiento = Agendamiento.fillna({'IDENTIFICACION': '0', 'NOMBRE': 'Sin Nombre','USUARIO': 'Sin identidad'})
    Agendamiento.columns = Agendamiento.columns.str.strip()
    Agendamiento['NOMBRE'] = [ele.rstrip(' ') for ele in Agendamiento['NOMBRE']]
    Agendamiento['NOMBRE'] = [ele.rstrip(' ') for ele in Agendamiento['NOMBRE']]

    Agendamiento['NOMBRE'] =Agendamiento['NOMBRE'].astype(str).str.replace("  ", " ", regex=True)
    Agendamiento = Agendamiento.sort_values('FECHA_CAMBIO', ascending=False)
    Agendamiento=Agendamiento[['IDENTIFICACION','NOMBRE','USUARIO','ESTADO']] #USAR SOLO ESAS TRES COLUMNAS
    Agendamiento.rename(columns={'IDENTIFICACION':'Documento', 'NOMBRE':'Nombre', 'USUARIO':'Identidad', 'ESTADO':'Estado'}, inplace=True)
    Agendamiento['Nombre'] =Agendamiento['Nombre'].astype(str).str.replace(".", "", regex=True)
    Agendamiento['Nombre']= Agendamiento["Nombre"].str.lower()
    Agendamiento['Identidad']= Agendamiento['Identidad'].str.lower()
    Agendamiento['Identidad'] = [ele.lstrip('.') for ele in Agendamiento['Identidad']]
    Agendamiento['Identidad'] = [ele.rstrip('.') for ele in Agendamiento['Identidad']]
    Agendamiento['Identidad'] = [ele.lstrip(' ') for ele in Agendamiento['Identidad']]
    Agendamiento['Identidad'] = [ele.rstrip(' ') for ele in Agendamiento['Identidad']]
    Agendamiento['Nombre'] = Agendamiento['Nombre'].replace('\s+', ' ', regex=True)
    Agendamiento['Nombre'] = Agendamiento['Nombre'].replace('\t', ' ', regex=True)
    Agendamiento['Nombre'] = Agendamiento['Nombre'].astype(str).str.replace("|", "", regex=True)
    Agendamiento['Nombre'] = Agendamiento['Nombre'].astype(str).str.replace(".", "", regex=True)
    Agendamiento['Nombre'] = Agendamiento['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Agendamiento['Nombre'] = Agendamiento['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Agendamiento['Nombre'] = Agendamiento['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Agendamiento['Nombre'] = Agendamiento['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Agendamiento['Nombre'] = Agendamiento['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Agendamiento['Nombre'] = Agendamiento['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Agendamiento['Nombre'] = [ele.lstrip(' ') for ele in Agendamiento['Nombre']]
    Agendamiento['Nombre'] = [ele.rstrip(' ') for ele in Agendamiento['Nombre']]
    
    Agendamiento['Estado'] = Agendamiento['Estado'].astype(str).str.replace("I", "Inactivo", regex=True)
    Agendamiento['Estado'] = Agendamiento['Estado'].astype(str).str.replace("E", "Inactivo", regex=True)
    Agendamiento['Estado'] = Agendamiento['Estado'].astype(str).str.replace("X", "Inactivo", regex=True)
    Agendamiento['Estado'] = Agendamiento['Estado'].astype(str).str.replace("A", "Activo", regex=True)
    Agendamiento['Estado'] = Agendamiento['Estado'].astype(str).str.replace("1", "N/A", regex=True)

    Agendamiento['Documento'] = Agendamiento['Documento'].astype(str).str.replace(".", "", regex=True)
    Agendamiento['Documento'] = Agendamiento['Documento'].astype(str).str.replace("?", "", regex=True) 
    Agendamiento['Documento'] = Agendamiento['Documento'].astype(str).str.replace("X", "", regex=True)
    Agendamiento['Documento'] = Agendamiento['Documento'].astype(str).str.replace("x", "", regex=True)

    Agendamiento['Documento'] = [ele.lstrip('0') for ele in Agendamiento['Documento']]
    Agendamiento['Documento'] = [ele.lstrip(' ') for ele in Agendamiento['Documento']]
    Agendamiento['Documento'] = [ele.rstrip(' ') for ele in Agendamiento['Documento']]
    Agendamiento['Identidad'] = [ele.rstrip(' ') for ele in Agendamiento['Identidad']]
    Agendamiento= Agendamiento.drop_duplicates(Agendamiento.columns[~Agendamiento.columns.isin(['Estado', 'Nombre'])], keep='first')
    Agendamiento = Agendamiento.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
if (not Agendamientobd.empty == True) or (file_exists_AGENDAMIENTO ==True):
    #filtro1=Agendamiento[Agendamiento['Documento'].str.contains(documentos_de_busqueda)]
    filtro1 = Agendamiento[Agendamiento.Documento.isin(lista1)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro1=filtro1.assign(Cruce='Documento')
    filtro1=filtro1.assign(DocumentoRRHH=filtro1['Documento'])
    filtro5 = Agendamiento[Agendamiento.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True) 
    filtro2 = Agendamiento[Agendamiento.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Agendamiento[Agendamiento.Nombre.isin(lista3)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Agendamiento[Agendamiento.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Agendamiento[Agendamiento.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Agendamiento[Agendamiento.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    Agendamiento = pd.concat([filtro1, filtro5])
    Agendamiento = pd.concat([Agendamiento, filtro2])
    Agendamiento = pd.concat([Agendamiento, filtro3])
    Agendamiento = pd.concat([Agendamiento, filtro4])
    Agendamiento = pd.concat([Agendamiento, filtro6])
    Agendamiento = pd.concat([Agendamiento, filtro7])

    Agendamiento.index = range(Agendamiento.shape[0])
    Agendamiento=Agendamiento[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Agendamiento= Agendamiento.drop_duplicates(Agendamiento.columns[~Agendamiento.columns.isin(['Cruce', 'Documento'])], keep='first')
    Agendamiento=Agendamiento.assign(Usuario_Aplicativo='AGENDAMIENTO')
    Agendamiento.insert(loc = 5,column = 'Modulo',value = 'N/A')
    listAgendamiento = Agendamiento['Identidad'].to_list()
    if "sin identidad" in listAgendamiento:
        listAgendamiento.remove("sin identidad")
    
    lista5 = lista5 + listAgendamiento
    Agendamiento1=Agendamiento[['Identidad', 'DocumentoRRHH']]
    Agendamiento1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Agendamiento1])
    Nube5= Nube5.drop_duplicates( keep='first')
if Agendamiento.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['AGENDAMIENTO'],
    'Modulo':['N/A']}
    Agendamiento = pd.DataFrame(datos)
Nube = pd.concat([Nube, Agendamiento])
Nube.index = range(Nube.shape[0])

SGA

In [ ]:
file_exists_SGA = os.path.exists(filenameT+"\\SGA.xlsx")

print(file_exists_SGA)
Sga = pd.DataFrame()
if file_exists_SGA == True:

    Sga=pd.read_excel(filenameT+"\\SGA.xlsx") #IMPORTAR

    Sga=Sga[['CEDULA','USUARIO','NOMBRE', 'ESTADO']] #USAR SOLO ESAS TRES COLUMNAS
    Sga.rename(columns={'CEDULA':'Documento', 'USUARIO':'Identidad','NOMBRE':'Nombre', 'ESTADO':'Estado'}, inplace=True)
    Sga['Documento'] = Sga['Documento'].astype(str)
    Sga['Nombre'] =Sga['Nombre'].astype(str).str.replace("  ", " ", regex=True)
    Sga["Nombre"]= Sga["Nombre"].str.lower()
    Sga["Identidad"]= Sga["Identidad"].str.lower()
    Sga['Identidad'] = [ele.lstrip('.') for ele in Sga['Identidad']]
    Sga['Identidad'] = [ele.rstrip('.') for ele in Sga['Identidad']]
    Sga['Identidad'] = [ele.lstrip(' ') for ele in Sga['Identidad']]
    Sga['Identidad'] = [ele.rstrip(' ') for ele in Sga['Identidad']]
    Sga['Nombre'] = Sga['Nombre'].replace('\s+', ' ', regex=True)
    Sga['Nombre'] = Sga['Nombre'].replace('\t', ' ', regex=True)
    Sga['Nombre'] = Sga['Nombre'].astype(str).str.replace("|", "", regex=True)
    Sga['Nombre'] = Sga['Nombre'].astype(str).str.replace(".", "", regex=True)
    Sga['Nombre'] = Sga['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Sga['Nombre'] = Sga['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Sga['Nombre'] = Sga['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Sga['Nombre'] = Sga['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Sga['Nombre'] = Sga['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Sga['Nombre'] = Sga['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Sga['Nombre'] = [ele.lstrip(' ') for ele in Sga['Nombre']]
    Sga['Nombre'] = [ele.rstrip(' ') for ele in Sga['Nombre']]
    Sga['Documento'] = Sga['Documento'].astype(str).str.replace(".", "", regex=True)
    Sga['Documento'] = Sga['Documento'].astype(str).str.replace("?", "", regex=True) 
    Sga['Documento'] = Sga['Documento'].astype(str).str.replace("X", "", regex=True)
    Sga['Documento'] = Sga['Documento'].astype(str).str.replace("x", "", regex=True)
    Sga['Documento'] = [ele.lstrip(' ') for ele in Sga['Documento']]
    Sga['Documento'] = [ele.rstrip(' ') for ele in Sga['Documento']]
    Sga = Sga.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
if file_exists_SGA == True:
    filtro1 = Sga[Sga.Documento.isin(lista1)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro1=filtro1.assign(Cruce='Documento')
    filtro1=filtro1.assign(DocumentoRRHH=filtro1['Documento'])
    filtro5 = Sga[Sga.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True) 
    filtro2 = Sga[Sga.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Sga[Sga.Nombre.isin(lista3)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Sga[Sga.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Sga[Sga.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Sga[Sga.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    Sga = pd.concat([filtro1, filtro5])
    Sga = pd.concat([Sga, filtro2])
    Sga = pd.concat([Sga, filtro3])
    Sga = pd.concat([Sga, filtro4])
    Sga = pd.concat([Sga, filtro6])
    Sga = pd.concat([Sga, filtro7])
    Sga.index = range(Sga.shape[0])
    Sga=Sga[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Sga= Sga.drop_duplicates(Sga.columns[~Sga.columns.isin(['Cruce', 'Documento'])], keep='first')
    Sga=Sga.assign(Usuario_Aplicativo='SGA')
    Sga.insert(loc = 5,column = 'Modulo',value = 'N/A')
    listSga = Sga['Identidad'].to_list()
    
    if "sin identidad" in listSga:
        listSga.remove("sin identidad") 
    
    lista5 = lista5 + listSga

    Sga1=Sga[['Identidad', 'DocumentoRRHH']]
    Sga1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Sga1])
    Nube5= Nube5.drop_duplicates( keep='first')
if Sga.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['SGA'],
    'Modulo':['N/A']}
    Sga = pd.DataFrame(datos)
Nube = pd.concat([Nube, Sga])
Nube.index = range(Nube.shape[0])

In [ ]:
Sga

SGA APEX

In [ ]:
file_exists_SGA_APEX = os.path.exists(filenameT+"\\SGA_APEX.xlsx")

print(file_exists_SGA_APEX)
Sga = pd.DataFrame()
if file_exists_SGA_APEX == True:

    SgaApex=pd.read_excel(filenameT+"\\SGA_APEX.xlsx") #IMPORTAR

    SgaApex=SgaApex[['USUARIO','NOMBRE', 'ESTADO']] #USAR SOLO ESAS TRES COLUMNAS
    SgaApex.rename(columns={'USUARIO':'Identidad','NOMBRE':'Nombre', 'ESTADO':'Estado'}, inplace=True)
    SgaApex['Nombre'] =SgaApex['Nombre'].astype(str).str.replace("  ", " ", regex=True)
    SgaApex["Nombre"]= SgaApex["Nombre"].str.lower()
    SgaApex["Identidad"]= SgaApex["Identidad"].str.lower()
    SgaApex['Identidad'] = [ele.lstrip('.') for ele in SgaApex['Identidad']]
    SgaApex['Identidad'] = [ele.rstrip('.') for ele in SgaApex['Identidad']]
    SgaApex['Identidad'] = [ele.lstrip(' ') for ele in SgaApex['Identidad']]
    SgaApex['Identidad'] = [ele.rstrip(' ') for ele in SgaApex['Identidad']]
    SgaApex['Nombre'] = SgaApex['Nombre'].replace('\s+', ' ', regex=True)
    SgaApex['Nombre'] = SgaApex['Nombre'].replace('\t', ' ', regex=True)
    SgaApex['Nombre'] = SgaApex['Nombre'].astype(str).str.replace("|", "", regex=True)
    SgaApex['Nombre'] = SgaApex['Nombre'].astype(str).str.replace(".", "", regex=True)
    SgaApex['Nombre'] = SgaApex['Nombre'].astype(str).str.replace("á", "a", regex=True)
    SgaApex['Nombre'] = SgaApex['Nombre'].astype(str).str.replace("é", "e", regex=True)
    SgaApex['Nombre'] = SgaApex['Nombre'].astype(str).str.replace("í", "i", regex=True)
    SgaApex['Nombre'] = SgaApex['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    SgaApex['Nombre'] = SgaApex['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    SgaApex['Nombre'] = SgaApex['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    SgaApex['Nombre'] = [ele.lstrip(' ') for ele in SgaApex['Nombre']]
    SgaApex['Nombre'] = [ele.rstrip(' ') for ele in SgaApex['Nombre']]
    SgaApex = SgaApex.fillna({ 'Identidad': 'sin identidad'})
    SgaApex.insert(loc = 0,column = 'Documento',value = '0')

In [ ]:
if file_exists_SGA_APEX == True:
    filtro5 = SgaApex[SgaApex.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True) 
    filtro2 = SgaApex[SgaApex.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = SgaApex[SgaApex.Nombre.isin(lista3)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = SgaApex[SgaApex.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = SgaApex[SgaApex.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = SgaApex[SgaApex.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    SgaApex = pd.concat([filtro5, filtro2])
    SgaApex = pd.concat([SgaApex, filtro3])
    SgaApex = pd.concat([SgaApex, filtro4])
    SgaApex = pd.concat([SgaApex, filtro6])
    SgaApex = pd.concat([SgaApex, filtro7])
    SgaApex.index = range(SgaApex.shape[0])
    SgaApex=SgaApex[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    SgaApex= SgaApex.drop_duplicates(SgaApex.columns[~SgaApex.columns.isin(['Cruce', 'Documento'])], keep='first')
    SgaApex=SgaApex.assign(Usuario_Aplicativo='SGA')
    SgaApex.insert(loc = 5,column = 'Modulo',value = 'APEX')
    listSgaApex = SgaApex['Identidad'].to_list()
    
    if "sin identidad" in listSgaApex:
        listSgaApex.remove("sin identidad") 
    
    lista5 = lista5 + listSgaApex

    SgaApex1=SgaApex[['Identidad', 'DocumentoRRHH']]
    SgaApex1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,SgaApex1])
    Nube5= Nube5.drop_duplicates( keep='first')
if SgaApex.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['SGA'],
    'Modulo':['APEX']}
    SgaApex = pd.DataFrame(datos)
Nube = pd.concat([Nube, SgaApex])
Nube.index = range(Nube.shape[0])

In [ ]:
SgaApex

In [ ]:
Nube

In [ ]:
Myit = pd.DataFrame()
Myitbd=ConectarM()

In [ ]:
# file_exists_MyIt = os.path.exists(filenameT+"\\MyIt.csv")

# print(file_exists_MyIt)
# Myit = pd.DataFrame()
if not Myitbd.empty == True:

    #Myit=pd.read_csv(filenameT+"\\MyIt.csv", sep=';',encoding = "utf-8" , engine='python') #IMPORTAR
    Myit= Myitbd[['CORPORATE_ID','REMEDY_LOGIN_ID','FIRST_NAME',  'MIDDLE_INITIAL',  'LAST_NAME', 'PROFILE_STATUS']]
    #Myit=Myit[['CORPORATE_ID','REMEDY_LOGIN_ID','FIRST_NAME',  'MIDDLE_INITIAL',  'LAST_NAME', 'PROFILE_STATUS']] #USAR SOLO ESAS TRES COLUMNAS
    Myit.rename(columns={'CORPORATE_ID':'Documento', 'REMEDY_LOGIN_ID':'Identidad', 'PROFILE_STATUS':'Estado'}, inplace=True)
    Myit = Myit.fillna({'FIRST_NAME': '0', 'MIDDLE_INITIAL': '0','LAST_NAME': '0', 'Documento':'0', 'Identidad':'sin Identidad'})
    Myit['Nombre'] = Myit["FIRST_NAME"] + " " + Myit["MIDDLE_INITIAL"]+ " " + Myit["LAST_NAME"]
    Myit['Nombre'] =Myit['Nombre'].astype(str).str.replace("0", "", regex=True)
    Myit['Nombre'] =Myit['Nombre'].astype(str).str.replace("  ", " ", regex=True)
    Myit=Myit[['Documento','Identidad','Nombre',  'Estado']] 
    Myit["Nombre"]= Myit["Nombre"].str.lower()
    Myit['Identidad'] = Myit['Identidad'].astype(str)
    Myit["Identidad"]= Myit["Identidad"].str.lower()
    Myit['Identidad'] = [ele.lstrip(' ') for ele in Myit['Identidad']]
    Myit['Identidad'] = [ele.rstrip(' ') for ele in Myit['Identidad']]
    Myit['Identidad'] = [ele.lstrip('.') for ele in Myit['Identidad']]
    Myit['Identidad'] = [ele.rstrip('.') for ele in Myit['Identidad']]
    Myit['Nombre'] = Myit['Nombre'].replace('\s+', ' ', regex=True)
    Myit['Nombre'] = Myit['Nombre'].replace('\t', ' ', regex=True)
    Myit['Nombre'] = Myit['Nombre'].astype(str).str.replace("|", "", regex=True)
    Myit['Nombre'] = Myit['Nombre'].astype(str).str.replace(".", "", regex=True)
    Myit['Nombre'] = Myit['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Myit['Nombre'] = Myit['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Myit['Nombre'] = Myit['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Myit['Nombre'] = Myit['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Myit['Nombre'] = Myit['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Myit['Nombre'] = Myit['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Myit['Nombre'] = [ele.lstrip(' ') for ele in Myit['Nombre']]
    Myit['Nombre'] = [ele.rstrip(' ') for ele in Myit['Nombre']]
    Myit['Documento'] = Myit['Documento'].astype(str).str.replace(".", "", regex=True)
    Myit['Documento'] = Myit['Documento'].astype(str).str.replace("?", "", regex=True) 
    Myit['Documento'] = Myit['Documento'].astype(str).str.replace("X", "", regex=True)
    Myit['Documento'] = Myit['Documento'].astype(str).str.replace("x", "", regex=True)
    Myit['Documento'] = [ele.lstrip(' ') for ele in Myit['Documento']]
    Myit['Documento'] = [ele.rstrip(' ') for ele in Myit['Documento']]
    Myit=Myit.replace({"Sin conexión": "Inactivo", "Activado": "Activo", "Eliminar":"Inactivo"})
    Myit = Myit.sort_values('Estado') 
    Myit = Myit.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
if not Myitbd.empty == True:
    #filtro1=Myit[Myit['Documento'].str.contains(documentos_de_busqueda)]
    filtro1 = Myit[Myit.Documento.isin(lista1)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro1=filtro1.assign(Cruce='Documento')
    filtro1=filtro1.assign(DocumentoRRHH=filtro1['Documento'])
    filtro5 = Myit[Myit.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True) 
    filtro2 = Myit[Myit.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Myit[Myit.Nombre.isin(lista3)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Myit[Myit.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Myit[Myit.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Myit[Myit.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    Myit = pd.concat([filtro1, filtro5])
    Myit = pd.concat([Myit, filtro2])
    Myit = pd.concat([Myit, filtro3])
    Myit = pd.concat([Myit, filtro4])
    Myit = pd.concat([Myit, filtro6])
    Myit = pd.concat([Myit, filtro7])
    Myit.index = range(Myit.shape[0])
    Myit=Myit[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Myit= Myit.drop_duplicates(Myit.columns[~Myit.columns.isin(['Cruce', 'Documento'])], keep='first')
    Myit=Myit.assign(Usuario_Aplicativo='MYIT')
    Myit.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listMyit = Myit['Identidad'].to_list()
    if "sin identidad" in listMyit:
        listMyit.remove("sin identidad")
    lista5 = lista5 + listMyit
    Myit1=Myit[['Identidad', 'DocumentoRRHH']]
    Myit1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Myit1])
    Nube5= Nube5.drop_duplicates( keep='first')

if Myit.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['MYIT'],
    'Modulo':['N/A']}
    Myit = pd.DataFrame(datos)
Nube = pd.concat([Nube, Myit])
Nube.index = range(Nube.shape[0])

IVALUA

In [ ]:
file_exists_Ivalua = os.path.exists(filenameT+"\\IVALUA.xlsx")

print(file_exists_Ivalua)
Ivalua = pd.DataFrame()
if file_exists_Ivalua == True:

    Ivalua=pd.read_excel(filenameT+"\\IVALUA.xlsx") #IMPORTAR

    Ivalua=Ivalua[['Contacto','Usuario','Estado']] #USAR SOLO ESAS TRES COLUMNAS
    Ivalua.rename(columns={ 'Usuario':'Nombre'}, inplace=True)
    Id = Ivalua["Contacto"].str.split("@", n=1, expand=True)
    Id= pd.DataFrame(Id)
    Id.columns = ['Id', 'correo']

    Ivalua = pd.concat([Ivalua, Id], axis=1)
    Ivalua['Identidad'] = Ivalua['Id'].apply(lowercase_if_not_numeric)
    Ivalua['Nombre'] =Ivalua['Nombre'].astype(str).str.replace("  ", " ", regex=True)
    Ivalua["Nombre"]= Ivalua["Nombre"].str.lower()
    Ivalua['Identidad'] = [ele.lstrip('.') for ele in Ivalua['Identidad']]
    Ivalua['Identidad'] = [ele.rstrip('.') for ele in Ivalua['Identidad']]
    Ivalua['Identidad'] = [ele.lstrip(' ') for ele in Ivalua['Identidad']]
    Ivalua['Identidad'] = [ele.rstrip(' ') for ele in Ivalua['Identidad']]
    Ivalua['Nombre'] = Ivalua['Nombre'].replace('\s+', ' ', regex=True)
    Ivalua['Nombre'] = Ivalua['Nombre'].replace('\t', ' ', regex=True)
    Ivalua['Nombre'] = Ivalua['Nombre'].astype(str).str.replace("|", "", regex=True)
    Ivalua['Nombre'] = Ivalua['Nombre'].astype(str).str.replace(".", "", regex=True)
    Ivalua['Nombre'] = Ivalua['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Ivalua['Nombre'] = Ivalua['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Ivalua['Nombre'] = Ivalua['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Ivalua['Nombre'] = Ivalua['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Ivalua['Nombre'] = Ivalua['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Ivalua['Nombre'] = Ivalua['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Ivalua['Nombre'] = [ele.lstrip(' ') for ele in Ivalua['Nombre']]
    Ivalua['Nombre'] = [ele.rstrip(' ') for ele in Ivalua['Nombre']]
    Ivalua['Identidad'] = Ivalua['Identidad'].astype(str).str.replace("-", "", regex=True)
    Ivalua['Identidad'] = Ivalua['Identidad'].astype(str).str.replace("|", "", regex=True)
    Ivalua.insert(loc = 0,column = 'Documento',value = '0')
    Ivalua['Identidad'] = Ivalua['Identidad'].astype(str).str.replace("?", "", regex=True) 
    Ivalua = Ivalua.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
Ivalua

In [ ]:
Ivalua = Ivalua[['Documento', 'Identidad', 'Nombre', 'Estado']]
filtro1 = pd.DataFrame()
documentosIvalua= Ivalua[['Identidad']]
documentosIvalua=documentosIvalua['Identidad'].to_list()
if file_exists_Ivalua == True:    
    for n in lista1:
        if n in documentosIvalua:
            filtroDocumento=Ivalua[Ivalua['Identidad'].str.contains(n)]
            filtroDocumento=filtroDocumento.assign(Cruce='Documento')
            filtroDocumento=filtroDocumento.assign(DocumentoRRHH = n)
            filtro1=pd.concat([filtro1, filtroDocumento])
    filtro5 = Ivalua[Ivalua.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro2 = Ivalua[Ivalua.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Ivalua[Ivalua.Nombre.isin(lista3)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Ivalua[Ivalua.Nombre.isin(lista4)] #filtra llave5
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Ivalua[Ivalua.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Ivalua[Ivalua.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    Ivalua = pd.concat([ filtro1, filtro5])
    Ivalua = pd.concat([ Ivalua, filtro2])
    Ivalua = pd.concat([ Ivalua, filtro3])
    Ivalua = pd.concat([ Ivalua, filtro4])
    Ivalua = pd.concat([ Ivalua, filtro6])
    Ivalua = pd.concat([ Ivalua, filtro7])
    Ivalua=Ivalua[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Ivalua= Ivalua.drop_duplicates(Ivalua.columns[~Ivalua.columns.isin(['Cruce'])], keep='first')
    Ivalua.index = range(Ivalua.shape[0])
    Ivalua=Ivalua.assign(Usuario_Aplicativo='IVALUA')
    Ivalua.insert(loc = 5,column = 'Modulo',value = 'N/A')
    
    listIvalua = Ivalua['Identidad'].to_list()
    if "sin identidad" in listIvalua:
        listIvalua.remove("sin identidad")
     
    lista5 = lista5 + listIvalua

    Ivalua1=Ivalua[['Identidad', 'DocumentoRRHH']]
    Ivalua1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Ivalua1])
    Nube5= Nube5.drop_duplicates( keep='first')

if Ivalua.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['IVALUA'],
    'Modulo':['N/A']}
    Ivalua = pd.DataFrame(datos)

Nube = pd.concat([Nube, Ivalua])
Nube.index = range(Nube.shape[0])

In [ ]:
Nube

CRM - ONIX

In [ ]:
file_exists_CRM = os.path.exists(filenameT+"\\CRM.csv")

print(file_exists_CRM)
Crm = pd.DataFrame()
if file_exists_CRM == True:

    Crm=pd.read_csv(filenameT+"\\CRM.csv", sep=';',encoding = "utf-8" , engine='python') #IMPORTAR
    Crm=Crm[['Cedula','UsuarioAplicacion','NombreCompleto', 'EstadoCuenta']] #USAR SOLO ESAS TRES COLUMNAS
    Crm.rename(columns={'Cedula':'Documento', 'UsuarioAplicacion':'Identidad','NombreCompleto':'Nombre', 'EstadoCuenta':'Estado'}, inplace=True)
    Crm['Documento'] = Crm['Documento'].astype(str)
    Crm['Nombre'] =Crm['Nombre'].astype(str).str.replace(".", "", regex=True)
    Crm['Nombre'] =Crm['Nombre'].astype(str).str.replace("  ", " ", regex=True)
    Crm["Nombre"]= Crm["Nombre"].str.lower()
    Crm["Identidad"]= Crm["Identidad"].str.lower()
    Crm['Identidad'] = [ele.lstrip('.') for ele in Crm['Identidad']]
    Crm['Identidad'] = [ele.rstrip('.') for ele in Crm['Identidad']]
    Crm['Identidad'] = [ele.lstrip(' ') for ele in Crm['Identidad']]
    Crm['Identidad'] = [ele.rstrip(' ') for ele in Crm['Identidad']]
    Crm['Nombre'] = Crm['Nombre'].replace('\s+', ' ', regex=True)
    Crm['Nombre'] = Crm['Nombre'].replace('\t', ' ', regex=True)
    Crm['Nombre'] = Crm['Nombre'].astype(str).str.replace("|", "", regex=True)
    Crm['Nombre'] = Crm['Nombre'].astype(str).str.replace(".", "", regex=True)
    Crm['Nombre'] = Crm['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Crm['Nombre'] = Crm['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Crm['Nombre'] = Crm['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Crm['Nombre'] = Crm['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Crm['Nombre'] = Crm['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Crm['Nombre'] = Crm['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Crm['Nombre'] = [ele.lstrip(' ') for ele in Crm['Nombre']]
    Crm['Nombre'] = [ele.rstrip(' ') for ele in Crm['Nombre']]
    Crm['Documento'] = Crm['Documento'].astype(str).str.replace(".", "", regex=True)
    Crm['Documento'] = Crm['Documento'].astype(str).str.replace("?", "", regex=True) 
    Crm['Documento'] = Crm['Documento'].astype(str).str.replace("X", "", regex=True)
    Crm['Documento'] = Crm['Documento'].astype(str).str.replace("x", "", regex=True)
    Crm['Documento'] = [ele.lstrip(' ') for ele in Crm['Documento']]
    Crm['Documento'] = [ele.rstrip(' ') for ele in Crm['Documento']]
    Crm = Crm.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
if file_exists_CRM == True:    
    filtro1 = Crm[Crm.Documento.isin(lista1)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro1=filtro1.assign(Cruce='Documento')
    filtro1=filtro1.assign(DocumentoRRHH=filtro1['Documento'])
    filtro5 = Crm[Crm.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro2 = Crm[Crm.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Crm[Crm.Nombre.isin(lista3)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Crm[Crm.Nombre.isin(lista4)] #filtra llave5
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Crm[Crm.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Crm[Crm.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    Crm = pd.concat([ filtro1, filtro5])
    Crm = pd.concat([ Crm, filtro2])
    Crm = pd.concat([ Crm, filtro3])
    Crm = pd.concat([ Crm, filtro4])
    Crm = pd.concat([ Crm, filtro6])
    Crm = pd.concat([ Crm, filtro7])
    Crm=Crm[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Crm= Crm.drop_duplicates(Crm.columns[~Crm.columns.isin(['Cruce'])], keep='first')
    Crm.index = range(Crm.shape[0])
    Crm=Crm.assign(Usuario_Aplicativo='CRM-ONYX')
    Crm.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listCrm = Crm['Identidad'].to_list()
    if "sin identidad" in listCrm:
        listCrm.remove("sin identidad") 
    lista5 = lista5 + listCrm

    Crm1=Crm[['Identidad', 'DocumentoRRHH']]
    Crm1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Crm1])
    Nube5= Nube5.drop_duplicates( keep='first')
if Crm.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['CRM-ONYX'],
    'Modulo':['N/A']}
    Crm = pd.DataFrame(datos)

Nube = pd.concat([Nube, Crm])
Nube.index = range(Nube.shape[0])

Motor de pagos

In [ ]:
file_exists_Motor = os.path.exists(filenameT+"\\MOTORDEPAGOS.csv")

print(file_exists_Motor)
Motor = pd.DataFrame()
if file_exists_Motor == True:

    Motor=pd.read_csv(filenameT+"\\MOTORDEPAGOS.csv", sep=',', encoding = "ISO-8859-1", names=['Usuario_Aplicacion','nombre','FECHA_ULT_INGRESO','FECHA_CREACION',
                                                                                               'ESTADO','ROL', 'ROL2']) #IMPORTAR
    Motor=Motor[['Usuario_Aplicacion','nombre', 'ESTADO']] #USAR SOLO ESAS TRES COLUMNAS
    Motor.insert(loc = 0,column = 'Documento',value = '0')
    Motor.rename(columns={'Usuario_Aplicacion':'Identidad','nombre':'Nombre', 'ESTADO':'Estado'}, inplace=True)
    Motor['Nombre'] =Motor['Nombre'].astype(str).str.replace(".", "", regex=True)
    Motor['Nombre'] =Motor['Nombre'].astype(str).str.replace("  ", " ", regex=True)
    Motor["Nombre"]= Motor["Nombre"].str.lower()
    Motor["Identidad"]= Motor["Identidad"].str.lower()
    Motor['Identidad'] = [ele.lstrip('.') for ele in Motor['Identidad']]
    Motor['Identidad'] = [ele.rstrip('.') for ele in Motor['Identidad']]
    Motor['Identidad'] = [ele.lstrip(' ') for ele in Motor['Identidad']]
    Motor['Identidad'] = [ele.rstrip(' ') for ele in Motor['Identidad']]
    Motor['Nombre'] = Motor['Nombre'].replace('\s+', ' ', regex=True)
    Motor['Nombre'] = Motor['Nombre'].replace('\t', ' ', regex=True)
    Motor['Nombre'] = Motor['Nombre'].astype(str).str.replace("|", "", regex=True)
    Motor['Nombre'] = Motor['Nombre'].astype(str).str.replace(".", "", regex=True)
    Motor['Nombre'] = Motor['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Motor['Nombre'] = Motor['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Motor['Nombre'] = Motor['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Motor['Nombre'] = Motor['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Motor['Nombre'] = Motor['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Motor['Nombre'] = Motor['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Motor['Nombre'] = [ele.lstrip(' ') for ele in Motor['Nombre']]
    Motor['Nombre'] = [ele.rstrip(' ') for ele in Motor['Nombre']]
    Motor = Motor.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
Motor

In [ ]:
if file_exists_Motor == True:

    filtro5 = Motor[Motor.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro2 = Motor[Motor.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Motor[Motor.Nombre.isin(lista3)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Motor[Motor.Nombre.isin(lista4)] #filtra llave5
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Motor[Motor.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Motor[Motor.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)


    Motor = pd.concat([filtro5, filtro2])
    Motor = pd.concat([Motor, filtro3])
    Motor = pd.concat([Motor, filtro4])
    Motor = pd.concat([Motor, filtro6])
    Motor = pd.concat([Motor, filtro7])
    Motor=Motor[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Motor= Motor.drop_duplicates(Motor.columns[~Motor.columns.isin(['Cruce'])], keep='first')
    Motor.index = range(Motor.shape[0])
    Motor=Motor.assign(Usuario_Aplicativo='MOTOR DE PAGOS')
    Motor.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listMotor = Motor['Identidad'].to_list()

    if "sin identidad" in listMotor:
        listMotor.remove("sin identidad") 
    lista5 = lista5 + listMotor

    Motor1=Motor[['Identidad', 'DocumentoRRHH']]
    Motor1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Motor1])
    Nube5= Nube5.drop_duplicates( keep='first')
if Motor.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['MOTOR DE PAGOS'],
    'Modulo':['N/A']}
    Motor = pd.DataFrame(datos)
    
Nube = pd.concat([Nube, Motor])
Nube.index = range(Nube.shape[0])

In [ ]:
Nube

SAP 6.0


In [ ]:
file_exists_sap = os.path.exists(filenameT+"\\SAP.xlsx")

print(file_exists_sap)
Sap = pd.DataFrame()
if file_exists_sap == True:

    Sap=pd.read_excel(filenameT+"\\SAP.xlsx") #IMPORTAR
    Sap.columns = Sap.columns.str.strip()
    try:
        Sap=Sap[['Cedula','Usuario',  'Estado']] #USAR SOLO ESAS TRES COLUMNAS
        Sap.rename(columns={'Cedula':'Documento', 'Usuario':'Identidad'}, inplace=True)
    except:
        print("usó USUARIOS")
        Sap=Sap[['Cedula','Usuarios',  'Estado']] #USAR SOLO ESAS TRES COLUMNAS
        Sap.rename(columns={'Cedula':'Documento', 'Usuarios':'Identidad'}, inplace=True)
    Sap['Identidad'] = Sap['Identidad'].astype(str)
    Sap["Identidad"]= Sap["Identidad"].str.lower()
    Sap['Identidad'] = [ele.lstrip('.') for ele in Sap['Identidad']]
    Sap['Identidad'] = [ele.rstrip('.') for ele in Sap['Identidad']]
    Sap['Identidad'] = [ele.lstrip(' ') for ele in Sap['Identidad']]
    Sap['Identidad'] = [ele.rstrip(' ') for ele in Sap['Identidad']]
    
    Sap.insert(loc = 2,column = 'Nombre',value = 'Sin Nombre')
    Sap['Documento'] = Sap['Documento'].astype(str).str.replace(".", "", regex=True)
    Sap['Documento'] = Sap['Documento'].astype(str).str.replace("?", "", regex=True) 
    Sap['Documento'] = Sap['Documento'].astype(str).str.replace("X", "", regex=True)
    Sap['Documento'] = Sap['Documento'].astype(str).str.replace("x", "", regex=True)

    Sap['Documento'] = [ele.lstrip('0') for ele in Sap['Documento']]
    Sap['Documento'] = [ele.lstrip(' ') for ele in Sap['Documento']]
    Sap['Documento'] = [ele.rstrip(' ') for ele in Sap['Documento']]
    Sap = Sap.fillna({ 'Identidad': 'sin identidad'})


In [ ]:
Sap

In [ ]:
if file_exists_sap == True:    
    #filtro1=Sap[Sap['Documento'].str.contains(documentos_de_busqueda)]
    filtro1 = Sap[Sap.Documento.isin(lista1)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro1=filtro1.assign(Cruce='Documento')
    filtro1=filtro1.assign(DocumentoRRHH=filtro1['Documento'])
    filtro5 = Sap[Sap.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad')
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    Sap = pd.concat([filtro1, filtro5])
    Sap=Sap[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Sap= Sap.drop_duplicates(Sap.columns[~Sap.columns.isin(['Cruce'])], keep='first')
    Sap=Sap.assign(Usuario_Aplicativo='SAP 6.0')
    Sap.insert(loc = 5,column = 'Modulo',value = 'N/A')


    listSap = Sap['Identidad'].to_list()

    if "sin identidad" in listSap:
        listSap.remove("sin identidad") 
    lista5 = lista5 + listSap

    Sap1=Sap[['Identidad', 'DocumentoRRHH']]
    Sap1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Sap1])
    Nube5= Nube5.drop_duplicates( keep='first')
if Sap.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['SAP 6.0'],
    'Modulo':['N/A']}
    Sap = pd.DataFrame(datos)
Nube = pd.concat([Nube, Sap])
Nube.index = range(Nube.shape[0])

In [ ]:
Nube

SICACOM

In [ ]:
file_exists_Sicacom1 = os.path.exists(filenameT+"\\oriente.txt")

print(file_exists_Sicacom1)
Sicacom = pd.DataFrame()
if file_exists_Sicacom1 == True:

    Sicacom1=pd.read_csv(filenameT+ "\\oriente.txt", sep=";", encoding="latin-1", engine="python", skiprows=1, header=None) #IMPORTAR
    Sicacom1 = Sicacom1.fillna({'0': '1'})
    
####
    file_exists_Sicacom2 = os.path.exists(filenameT+"\\occidente.txt")

    print(file_exists_Sicacom2)
    if file_exists_Sicacom2 == True:

        Sicacom2=pd.read_csv(filenameT+ "\\occidente.txt", sep=";", encoding="latin-1", engine="python", skiprows=1, header=None) #IMPORTAR
        Sicacom2 = Sicacom2.fillna({'0': '2'})
    ####
        file_exists_Sicacom3 = os.path.exists(filenameT+"\\costa.txt")

        print(file_exists_Sicacom3)
        if file_exists_Sicacom3 == True:

            Sicacom3=pd.read_csv(filenameT+ "\\costa.txt", sep=";", encoding="latin-1", engine="python", skiprows=1, header=None) #IMPORTAR
            Sicacom3 = Sicacom3.fillna({'0': '3'})

            Sicacom = pd.concat([Sicacom1, Sicacom2, Sicacom3], axis=0)

In [ ]:
if not Sicacom.empty == True:
    
    Sicacom.rename(columns={9:'Identidad', 5:'Documento',3:'Nombre', 0:'Modulo', 10:'Estado'}, inplace=True)
    Sicacom=Sicacom[['Identidad','Documento','Nombre','Modulo','Estado']]
    Sicacom.fillna({'Documento' : '0','Identidad': 'sin identidad'})

    Sicacom['Modulo'] = Sicacom['Modulo'].astype(str).str.replace("1", "Oriente", regex=True)
    Sicacom['Modulo'] = Sicacom['Modulo'].astype(str).str.replace("2", "Occidente", regex=True)
    Sicacom['Modulo'] = Sicacom['Modulo'].astype(str).str.replace("3", "Costa", regex=True)

    
    Sicacom['Estado'] = Sicacom['Estado'].astype(str).str.replace("I", "Inactivo", regex=True)
    
    Sicacom['Estado'] = Sicacom['Estado'].astype(str).str.replace("A", "Activo", regex=True)
    Sicacom['Estado'] = Sicacom['Estado'].astype(str).str.replace("T", "Activo", regex=True)
    Sicacom['Estado'] = Sicacom['Estado'].astype(str).str.replace("B", "Activo", regex=True)
    Sicacom = Sicacom.sort_values('Estado')    

    Sicacom["Identidad"]= Sicacom["Identidad"].str.lower()
    Sicacom["Nombre"]= Sicacom["Nombre"].str.lower()

    Sicacom['Nombre'] = Sicacom['Nombre'].replace('\s+', ' ', regex=True)
    Sicacom['Nombre'] = Sicacom['Nombre'].replace('\t', ' ', regex=True)
    Sicacom['Nombre'] = Sicacom['Nombre'].astype(str).str.replace("|", "", regex=True)
    Sicacom['Nombre'] = Sicacom['Nombre'].astype(str).str.replace(".", "", regex=True)
    Sicacom['Nombre'] = Sicacom['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Sicacom['Nombre'] = Sicacom['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Sicacom['Nombre'] = Sicacom['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Sicacom['Nombre'] = Sicacom['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Sicacom['Nombre'] = Sicacom['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Sicacom['Nombre'] = Sicacom['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Sicacom['Identidad'] = [ele.lstrip(' ') for ele in Sicacom['Identidad']]
    Sicacom['Identidad'] = [ele.rstrip(' ') for ele in Sicacom['Identidad']]
    Sicacom['Identidad'] = [ele.lstrip('.') for ele in Sicacom['Identidad']]
    Sicacom['Identidad'] = [ele.rstrip('.') for ele in Sicacom['Identidad']]
    
    Sicacom['Documento'] = Sicacom['Documento'].astype(str).str.replace("A", "1", regex=True)
    Sicacom['Documento'] = Sicacom['Documento'].astype(str).str.replace("B", "2", regex=True)
    Sicacom['Documento'] = Sicacom['Documento'].astype(str).str.replace("C", "3", regex=True)
    Sicacom['Documento'] = Sicacom['Documento'].astype(str).str.replace("D", "4", regex=True)
    Sicacom['Documento'] = Sicacom['Documento'].astype(str).str.replace("E", "5", regex=True)
    Sicacom['Documento'] = Sicacom['Documento'].astype(str).str.replace("F", "6", regex=True)
    Sicacom['Documento'] = Sicacom['Documento'].astype(str).str.replace("G", "7", regex=True)
    Sicacom['Documento'] = Sicacom['Documento'].astype(str).str.replace("H", "8", regex=True)
    Sicacom['Documento'] = Sicacom['Documento'].astype(str).str.replace("I", "9", regex=True)
    Sicacom['Documento'] = [ele.lstrip(' ') for ele in Sicacom['Documento']]
    Sicacom['Documento'] = [ele.rstrip(' ') for ele in Sicacom['Documento']]
    Sicacom['Estado'] = [ele.lstrip(' ') for ele in Sicacom['Estado']]
    Sicacom['Estado'] = [ele.rstrip(' ') for ele in Sicacom['Estado']]
    Sicacom['Documento'] = [ele.lstrip('.') for ele in Sicacom['Documento']]
    Sicacom['Documento'] = [ele.rstrip('.') for ele in Sicacom['Documento']]
    
    Sicacom['Nombre'] = [ele.lstrip(' ') for ele in Sicacom['Nombre']]
    Sicacom['Nombre'] = [ele.rstrip(' ') for ele in Sicacom['Nombre']]


In [ ]:
if not Sicacom.empty:
    #filtro1=Sicacom[Sicacom['Documento'].str.contains(documentos_de_busqueda)]
    filtro1 = Sicacom[Sicacom.Documento.isin(lista1)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro1=filtro1.assign(Cruce='Documento')
    filtro1=filtro1.assign(DocumentoRRHH=filtro1['Documento'])
    filtro5 = Sicacom[Sicacom.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad')
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True) 
    filtro2 = Sicacom[Sicacom.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Sicacom[Sicacom.Nombre.isin(lista3)]#filtra llave1
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Sicacom[Sicacom.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Sicacom[Sicacom.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Sicacom[Sicacom.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    
    Sicacom = pd.concat([ filtro1, filtro5])
    Sicacom = pd.concat([ Sicacom, filtro2])
    Sicacom = pd.concat([ Sicacom, filtro3])
    Sicacom = pd.concat([ Sicacom, filtro4])
    Sicacom = pd.concat([ Sicacom, filtro6])
    Sicacom = pd.concat([ Sicacom, filtro7])
    Sicacom.index = range(Sicacom.shape[0])
    Sicacom=Sicacom[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'Modulo','DocumentoRRHH']]
    Sicacom= Sicacom.drop_duplicates(Sicacom.columns[~Sicacom.columns.isin(['Cruce'])], keep='first')
    Sicacom=Sicacom.assign(Usuario_Aplicativo='SICACOM')

    listSicacom = Sicacom['Identidad'].to_list()

    if "sin identidad" in listSicacom:
        listSicacom.remove("sin identidad")
    lista5 = lista5 + listSicacom

    Sicacom1=Sicacom[['Identidad', 'DocumentoRRHH']]
    Sicacom1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Sicacom1])
    Nube5= Nube5.drop_duplicates( keep='first')
if Sicacom.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['SICACOM'],
    'Modulo':['N/A']}
    Sicacom = pd.DataFrame(datos)
    
Nube = pd.concat([Nube, Sicacom])
Nube.index = range(Nube.shape[0])

In [ ]:
Nube

AC

In [ ]:
Ac = pd.DataFrame()
Acbd=ConectarAC()

In [ ]:
file_exists_AC = os.path.exists(filenameT+"\\AC.xlsx")
if not Acbd.empty == True:

    #Ac=pd.read_excel(filenameT+"\\AC.xlsx", sheet_name=1) #IMPORTAR
    Ac=Acbd[['USUARIORED',  'NOMBRE',  'ACTIVO']] #USAR SOLO ESAS TRES COLUMNAS
    Ac.rename(columns={'USUARIORED':'Identidad','NOMBRE':'Nombre', 'ACTIVO':'Estado'}, inplace=True)
    Ac=Ac[['Identidad',  'Nombre',  'Estado']] #USAR SOLO ESAS TRES COLUMNAS
    Ac["Nombre"]= Ac["Nombre"].str.lower()
    Ac['Identidad'] = Ac['Identidad'].astype(str)
    Ac["Identidad"]= Ac["Identidad"].str.lower()
    Ac['Identidad'] = [ele.lstrip(' ') for ele in Ac['Identidad']]
    Ac['Identidad'] = [ele.rstrip(' ') for ele in Ac['Identidad']]
    Ac['Identidad'] = [ele.lstrip('.') for ele in Ac['Identidad']]
    Ac['Identidad'] = [ele.rstrip('.') for ele in Ac['Identidad']]
    Ac['Nombre'] = Ac['Nombre'].replace('\s+', ' ', regex=True)
    Ac['Nombre'] = Ac['Nombre'].replace('\t', ' ', regex=True)
    Ac['Nombre'] = Ac['Nombre'].astype(str).str.replace("|", "", regex=True)
    Ac['Nombre'] = Ac['Nombre'].astype(str).str.replace(".", "", regex=True)
    Ac['Nombre'] = Ac['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Ac['Nombre'] = Ac['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Ac['Nombre'] = Ac['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Ac['Nombre'] = Ac['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Ac['Nombre'] = Ac['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Ac['Nombre'] = Ac['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Ac['Nombre'] = [ele.lstrip(' ') for ele in Ac['Nombre']]
    Ac['Nombre'] = [ele.rstrip(' ') for ele in Ac['Nombre']]
    Ac['Estado'] = Ac['Estado'].astype(str).str.replace("X", "Activo", regex=True)
    Ac.loc[Ac['Estado'] != 'Activo','Estado']='Inactivo'
    Ac = Ac.sort_values('Estado')
    Ac= Ac.drop_duplicates(Ac.columns[~Ac.columns.isin(['Cruce'])], keep='first')
    Ac.insert(loc = 0,column = 'Documento',value = '0')
    Ac = Ac.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
if not Acbd.empty == True:
    filtro5 = Ac[Ac.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad')
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True) 
    filtro2 = Ac[Ac.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Ac[Ac.Nombre.isin(lista3)]#filtra llave1
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Ac[Ac.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Ac[Ac.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Ac[Ac.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    Ac = pd.concat([filtro5, filtro2])
    Ac = pd.concat([Ac, filtro3])
    Ac = pd.concat([Ac, filtro4])
    Ac = pd.concat([Ac, filtro6])
    Ac = pd.concat([Ac, filtro7])
    Ac.index = range(Ac.shape[0])
    Ac=Ac[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Ac= Ac.drop_duplicates(Ac.columns[~Ac.columns.isin(['Cruce'])], keep='first')
    Ac=Ac.assign(Usuario_Aplicativo='AC')
    Ac.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listAc = Ac['Identidad'].to_list()
    if "sin identidad" in listAc:
        listAc.remove("sin identidad")
    lista5 = lista5 + listAc
    Ac1=Ac[['Identidad', 'DocumentoRRHH']]
    Ac1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Ac1])
    Nube5= Nube5.drop_duplicates( keep='first')
if Ac.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['AC'],
    'Modulo':['N/A']}
    Ac = pd.DataFrame(datos)

Nube = pd.concat([Nube, Ac])
Nube.index = range(Nube.shape[0])

In [ ]:
Ac

In [ ]:
Nube

ASCARD

In [ ]:
file_exists_Ascard = os.path.exists(filenameT+"\\ASCARD.txt")

print(file_exists_Ascard)
Ascard = pd.DataFrame()
if file_exists_Ascard == True:

    Ascard=pd.read_csv(filenameT+"\\ASCARD.txt",sep='|', encoding='ISO-8859-1',  names=['Column1', 'Column2', '3', '4','5','Column6','7','8','Column9','10','11', '12', '13', '14','15','16','17','18','19','20']) #IMPORTAR
    Ascard=Ascard[['Column9','Column1',  'Column2',  'Column6']] #USAR SOLO ESAS TRES COLUMNAS
    Ascard.rename(columns={'Column9':'Documento','Column1':'Identidad','Column2':'Nombre', 'Column6':'Estado'}, inplace=True)
    Ascard["Nombre"]= Ascard["Nombre"].str.lower()
    Ascard['Identidad'] = Ascard['Identidad'].astype(str)
    Ascard["Identidad"]= Ascard["Identidad"].str.lower()
    Ascard['Identidad'] = [ele.lstrip(' ') for ele in Ascard['Identidad']]
    Ascard['Identidad'] = [ele.rstrip(' ') for ele in Ascard['Identidad']]
    Ascard['Identidad'] = [ele.lstrip('.') for ele in Ascard['Identidad']]
    Ascard['Identidad'] = [ele.rstrip('.') for ele in Ascard['Identidad']]
    Ascard['Nombre'] = Ascard['Nombre'].replace('\s+', ' ', regex=True)
    Ascard['Nombre'] = Ascard['Nombre'].replace('\t', ' ', regex=True)
    Ascard['Nombre'] = Ascard['Nombre'].astype(str).str.replace("|", "", regex=True)
    Ascard['Nombre'] = Ascard['Nombre'].astype(str).str.replace(".", "", regex=True)
    Ascard['Nombre'] = Ascard['Nombre'].astype(str).str.replace("  ", " ", regex=True)
    Ascard['Nombre'] = Ascard['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Ascard['Nombre'] = Ascard['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Ascard['Nombre'] = Ascard['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Ascard['Nombre'] = Ascard['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Ascard['Nombre'] = Ascard['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Ascard['Nombre'] = Ascard['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Ascard['Nombre'] = Ascard['Nombre'].astype(str).str.replace("?", "", regex=True)
    Ascard['Nombre'] = [ele.lstrip(' ') for ele in Ascard['Nombre']]
    Ascard['Nombre'] = [ele.rstrip(' ') for ele in Ascard['Nombre']]
    Ascard['Documento'] = Ascard['Documento'].astype(str)
    Ascard['Documento'] = Ascard['Documento'].astype(str).str.replace(".0", "", regex=True)
    Ascard['Documento'] = Ascard['Documento'].astype(str).str.replace(".", "", regex=True)
    Ascard['Documento'] = Ascard['Documento'].astype(str).str.replace("?", "", regex=True) 
    Ascard['Documento'] = Ascard['Documento'].astype(str).str.replace("X", "", regex=True)
    Ascard['Documento'] = Ascard['Documento'].astype(str).str.replace("x", "", regex=True)
    Ascard['Documento'] = [ele.lstrip(' ') for ele in Ascard['Documento']]
    Ascard['Documento'] = [ele.rstrip(' ') for ele in Ascard['Documento']]
    Ascard['Estado'] = Ascard['Estado'].astype(str).str.replace("I", "Inactivo", regex=True)
    Ascard.loc[Ascard['Estado'] != 'Inactivo','Estado']='Activo'
    Ascard = Ascard.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
Ascard

In [ ]:
if file_exists_Ascard == True:
    #filtro1=Ascard[Ascard['Documento'].str.contains(documentos_de_busqueda)]
    filtro1 = Ascard[Ascard.Documento.isin(lista1)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro1=filtro1.assign(Cruce='Documento')
    filtro1=filtro1.assign(DocumentoRRHH=filtro1['Documento'])
    filtro5 = Ascard[Ascard.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad')
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True) 
    filtro2 = Ascard[Ascard.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Ascard[Ascard.Nombre.isin(lista3)]#filtra llave1
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Ascard[Ascard.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Ascard[Ascard.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Ascard[Ascard.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    Ascard = pd.concat([filtro1, filtro5])
    Ascard = pd.concat([ Ascard, filtro2])
    Ascard = pd.concat([ Ascard, filtro3])
    Ascard = pd.concat([ Ascard, filtro4])
    Ascard = pd.concat([ Ascard, filtro6])
    Ascard = pd.concat([ Ascard, filtro7])
    Ascard.index = range(Ascard.shape[0])
    Ascard=Ascard[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Ascard= Ascard.drop_duplicates(Ascard.columns[~Ascard.columns.isin(['Cruce'])], keep='first')
    Ascard=Ascard.assign(Usuario_Aplicativo='ASCARD')
    Ascard.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listAscard = Ascard['Identidad'].to_list()
    if "sin identidad" in listAscard:
        listAscard.remove("sin identidad")
    lista5 = lista5 + listAscard
    Ascard1=Ascard[['Identidad', 'DocumentoRRHH']]
    Ascard1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Ascard1])
    Nube5= Nube5.drop_duplicates( keep='first')
if Ascard.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['ASCARD'],
    'Modulo':['N/A']}
    Ascard = pd.DataFrame(datos)

Nube = pd.concat([Nube, Ascard])
Nube.index = range(Nube.shape[0])

In [ ]:
Ascard

In [ ]:
Nube

CMAX


In [ ]:
file_exists_Cmax = os.path.exists(filenameT+"\\CMAX.csv")#Pregunta si en la ruta existe el archivo
x = list(range(31))
print(file_exists_Cmax) #muestra false o true 
Cmax = pd.DataFrame()
if file_exists_Cmax == True: #Si el archivo existe

    Cmax=pd.read_csv(filenameT+"\\CMAX.csv", sep=",", encoding="utf-8-sig", engine="python", names=x, skiprows=1)  #importa el archivo csv separado por punto y comas
    
    Cmax=Cmax[[0, 1, 21]] #crea un dataframe con solo la columa Documento 
    Cmax.rename(columns={0:'Identidad', 1:'Nombre', 21:'Estado'}, inplace=True)
    Cmax.insert(loc = 0,column = 'Documento',value = '0')
    Cmax = Cmax.fillna({'Estado': 'Activo'})
    Cmax.loc[Cmax['Estado'] != 'Activo','Estado']='Inactivo'
    Cmax["Nombre"]= Cmax["Nombre"].str.lower()
    Cmax['Identidad'] = Cmax['Identidad'].astype(str)
    Cmax['Nombre'] = Cmax['Nombre'].astype(str)
    Cmax["Identidad"]= Cmax["Identidad"].str.lower()
    Cmax['Identidad'] = [ele.lstrip(' ') for ele in Cmax['Identidad']]
    Cmax['Identidad'] = [ele.rstrip(' ') for ele in Cmax['Identidad']]
    Cmax['Identidad'] = [ele.lstrip('.') for ele in Cmax['Identidad']]
    Cmax['Identidad'] = [ele.rstrip('.') for ele in Cmax['Identidad']]
    Cmax['Nombre'] = Cmax['Nombre'].replace('\s+', ' ', regex=True)
    Cmax['Nombre'] = Cmax['Nombre'].replace('\t', ' ', regex=True)
    Cmax['Nombre'] = [ele.lstrip(' ') for ele in Cmax['Nombre']]
    Cmax['Nombre'] = [ele.rstrip(' ') for ele in Cmax['Nombre']]
    Cmax['Nombre'] = Cmax['Nombre'].astype(str).str.replace("|", "", regex=True)
    Cmax['Nombre'] = Cmax['Nombre'].astype(str).str.replace(".", "", regex=True)
    Cmax['Nombre'] = Cmax['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Cmax['Nombre'] = Cmax['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Cmax['Nombre'] = Cmax['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Cmax['Nombre'] = Cmax['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Cmax['Nombre'] = Cmax['Nombre'].astype(str).str.replace("ú", "u", regex=True)  
    Cmax['Nombre'] = Cmax['Nombre'].astype(str).str.replace("ñ", "n", regex=True)  
    Cmax = Cmax.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
Cmax

Filtro CMAX

In [ ]:
if file_exists_Cmax == True:
    filtro5 = Cmax[Cmax.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad')
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True) 
    filtro2 = Cmax[Cmax.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Cmax[Cmax.Nombre.isin(lista3)]#filtra llave1
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Cmax[Cmax.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Cmax[Cmax.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Cmax[Cmax.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    Cmax = pd.concat([filtro5, filtro2])
    Cmax = pd.concat([Cmax, filtro3])
    Cmax = pd.concat([Cmax, filtro4])
    Cmax = pd.concat([Cmax, filtro6])
    Cmax = pd.concat([Cmax, filtro7])
    Cmax.index = range(Cmax.shape[0])
    Cmax=Cmax[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Cmax= Cmax.drop_duplicates(Cmax.columns[~Cmax.columns.isin(['Cruce'])], keep='first')
    Cmax=Cmax.assign(Usuario_Aplicativo='CMAX')
    Cmax.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listCmax = Cmax['Identidad'].to_list()
    if "sin identidad" in listCmax:
        listCmax.remove("sin identidad")
    lista5 = lista5 + listCmax
    Cmax1=Cmax[['Identidad', 'DocumentoRRHH']]
    Cmax1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Cmax1])
    Nube5= Nube5.drop_duplicates( keep='first')
if Cmax.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['CMAX'],
    'Modulo':['N/A']}
    Cmax = pd.DataFrame(datos)

Nube = pd.concat([Nube, Cmax])
Nube.index = range(Nube.shape[0])

In [ ]:
Nube

SAP CLOUD

In [ ]:
file_exists_sapCloud = os.path.exists(filenameT+"\\SAP Payroll.xlsx")

print(file_exists_sapCloud)
Sappay = pd.DataFrame()
if file_exists_sapCloud == True:

    Sappay=pd.read_excel(filenameT+"\\SAP Payroll.xlsx", sheet_name=0) #IMPORTAR
    try:
        Sappay=Sappay[['Usuario',  'Cedula',  'Estado']] #USAR SOLO ESAS TRES COLUMNAS
        Sappay.rename(columns={'Cedula':'Documento', 'Usuario':'Identidad'}, inplace=True)
    except:
        print("ejecutó USUARIOS")
        Sappay=Sappay[['Usuarios',  'Cedula',  'Estado']] #USAR SOLO ESAS TRES COLUMNAS
        Sappay.rename(columns={'Cedula':'Documento', 'Usuarios':'Identidad'}, inplace=True)
    Sappay['Identidad'] = Sappay['Identidad'].astype(str)
    Sappay["Identidad"]= Sappay["Identidad"].str.lower()
    Sappay['Identidad'] = [ele.lstrip(' ') for ele in Sappay['Identidad']]
    Sappay['Identidad'] = [ele.rstrip(' ') for ele in Sappay['Identidad']]
    Sappay['Identidad'] = [ele.lstrip('.') for ele in Sappay['Identidad']]
    Sappay['Identidad'] = [ele.rstrip('.') for ele in Sappay['Identidad']]
    Sappay['Documento'] = Sappay['Documento'].astype(str).str.replace(".", "", regex=True)
    Sappay['Documento'] = Sappay['Documento'].astype(str).str.replace("?", "", regex=True) 
    Sappay['Documento'] = Sappay['Documento'].astype(str).str.replace("X", "", regex=True)
    Sappay['Documento'] = Sappay['Documento'].astype(str).str.replace("x", "", regex=True)
    Sappay['Documento'] = [ele.lstrip(' ') for ele in Sappay['Documento']]
    Sappay['Documento'] = [ele.rstrip(' ') for ele in Sappay['Documento']]
    Sappay['Documento'] = [ele.lstrip('0') for ele in Sappay['Documento']]
    Sappay.insert(loc = 2,column = 'Nombre',value = 'Sin Nombre')
    Sappay = Sappay.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
if file_exists_sapCloud == True:
    Sappay1= Sappay[['Documento',  'Identidad',  'Nombre', 'Estado']]
    #filtro1=Sappay1[Sappay1['Documento'].str.contains(documentos_de_busqueda)]
    filtro1 = Sappay1[Sappay1.Documento.isin(lista1)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro1=filtro1.assign(Cruce='Documento')
    filtro1=filtro1.assign(DocumentoRRHH=filtro1['Documento'])
    filtro5 = Sappay1[Sappay1.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad')
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True) 
    Sappay= pd.concat([filtro1, filtro5])
    Sappay.index = range(Sappay.shape[0])
    Sappay=Sappay[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Sappay= Sappay.drop_duplicates(Sappay.columns[~Sappay.columns.isin(['Cruce'])], keep='first')
    Sappay=Sappay.assign(Usuario_Aplicativo='SAP CLOUD')
    Sappay.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listSappay = Sappay['Identidad'].to_list()
    if "sin identidad" in listSappay:
        listSappay.remove("sin identidad")
    lista5 = lista5 + listSappay
    Sappay1=Sappay[['Identidad', 'DocumentoRRHH']]
    Sappay1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Sappay1])
    Nube5= Nube5.drop_duplicates( keep='first')

if Sappay.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['SAP CLOUD'],
    'Modulo':['N/A']}
    Sappay = pd.DataFrame(datos)
Nube = pd.concat([Nube, Sappay])
Nube.index = range(Nube.shape[0])

In [ ]:
Sappay

In [ ]:
Nube

In [ ]:
Nube

CALLIDUS

In [ ]:
file_exists_Callidus = os.path.exists(filenameT+"\\CALLIDUS.xlsx")

print(file_exists_Callidus)
Callidus = pd.DataFrame()
if file_exists_Callidus == True:

    Callidus=pd.read_excel(filenameT+"\\CALLIDUS.xlsx", sheet_name=0) #IMPORTAR
    Callidus=Callidus[['USUARIO','NOMBRE','ESTADO']] #USAR SOLO ESAS TRES COLUMNAS
    Callidus.rename(columns={'USUARIO':'Identidad','NOMBRE':'Nombre', 'ESTADO':'Estado'}, inplace=True)
    Callidus["Nombre"]= Callidus["Nombre"].str.lower()
    Callidus["Identidad"]= Callidus["Identidad"].str.lower()
    Callidus['Identidad'] = [ele.lstrip(' ') for ele in Callidus['Identidad']]
    Callidus['Identidad'] = [ele.rstrip(' ') for ele in Callidus['Identidad']]
    Callidus['Identidad'] = [ele.lstrip('.') for ele in Callidus['Identidad']]
    Callidus['Identidad'] = [ele.rstrip('.') for ele in Callidus['Identidad']]
    Callidus['Nombre'] = Callidus['Nombre'].replace('\s+', ' ', regex=True)
    Callidus['Nombre'] = Callidus['Nombre'].replace('\t', ' ', regex=True)
    Callidus['Nombre'] = Callidus['Nombre'].astype(str).str.replace("|", "", regex=True)
    Callidus['Nombre'] = Callidus['Nombre'].astype(str).str.replace(".", "", regex=True)
    Callidus['Nombre'] = Callidus['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Callidus['Nombre'] = Callidus['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Callidus['Nombre'] = Callidus['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Callidus['Nombre'] = Callidus['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Callidus['Nombre'] = Callidus['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Callidus['Nombre'] = Callidus['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Callidus['Estado'] = Callidus['Estado'].astype(str).str.replace("0", "Activo", regex=True)
    Callidus['Estado'] = Callidus['Estado'].astype(str).str.replace("1", "Inactivo", regex=True)
    Callidus['Nombre'] = [ele.lstrip(' ') for ele in Callidus['Nombre']]
    Callidus['Nombre'] = [ele.rstrip(' ') for ele in Callidus['Nombre']]
    Callidus.insert(loc = 0,column = 'Documento',value = '0')
    Callidus = Callidus.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
if file_exists_Callidus == True:
    filtro5 = Callidus[Callidus.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True) 
    filtro2 = Callidus[Callidus.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Callidus[Callidus.Nombre.isin(lista3)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Callidus[Callidus.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Callidus[Callidus.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Callidus[Callidus.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    Callidus = pd.concat([filtro5, filtro2])
    Callidus = pd.concat([Callidus, filtro3])
    Callidus = pd.concat([Callidus, filtro4])
    Callidus = pd.concat([Callidus, filtro6])
    Callidus = pd.concat([Callidus, filtro7])

    Callidus.index = range(Callidus.shape[0])
    Callidus=Callidus[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Callidus= Callidus.drop_duplicates(Callidus.columns[~Callidus.columns.isin(['Cruce'])], keep='first')
    Callidus=Callidus.assign(Usuario_Aplicativo='CALLIDUS')
    Callidus.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listCallidus = Callidus['Identidad'].to_list()
    if "sin identidad" in listCallidus:
        listCallidus.remove("sin identidad")
    lista5 = lista5 + listCallidus
    Callidus1=Callidus[['Identidad', 'DocumentoRRHH']]
    Callidus1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Callidus1])
    Nube5= Nube5.drop_duplicates( keep='first')

if Callidus.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['CALLIDUS'],
    'Modulo':['N/A']}
    Callidus = pd.DataFrame(datos)
Nube = pd.concat([Nube, Callidus])
Nube.index = range(Nube.shape[0])

In [ ]:
Nube

BSCS MODULOS

In [ ]:
midb=ConectarBD()
BscsMod=pd.DataFrame()

In [ ]:
midb

In [ ]:
# file_exists_ModBscs = os.path.exists(filenameT+"\\BSCS MODULOS.csv")

# print(file_exists_ModBscs)
if not midb.empty:

    # BscsMod=pd.read_csv(filenameT+"\\BSCS MODULOS.csv", sep=';',encoding = "utf-8" , engine='python') #IMPORTAR
    BscsMod=midb[['USERNAME','DESCRIPTION','ESTATUS_USUARIO_ORACLE',  'MODULO']] #USAR SOLO ESAS TRES COLUMNAS
    nom = BscsMod["DESCRIPTION"].str.split("-", n=2, expand=True)
    nom= pd.DataFrame(nom)
    nom.columns = ['1', '2', '3']

    BscsMod = pd.concat([BscsMod, nom], axis=1)
    BscsMod.rename(columns={'USERNAME':'Identidad', '1':'Nombre', 'ESTATUS_USUARIO_ORACLE':'Estado', 'MODULO':'Modulo'}, inplace=True)
    BscsMod=BscsMod[['Identidad','Nombre','Estado',  'Modulo']]
    BscsMod["Nombre"]= BscsMod["Nombre"].str.lower()
    BscsMod["Identidad"]= BscsMod["Identidad"].str.lower()
    BscsMod['Identidad'] = [ele.lstrip(' ') for ele in BscsMod['Identidad']]
    BscsMod['Identidad'] = [ele.rstrip(' ') for ele in BscsMod['Identidad']]
    BscsMod['Identidad'] = [ele.lstrip('.') for ele in BscsMod['Identidad']]
    BscsMod['Identidad'] = [ele.rstrip('.') for ele in BscsMod['Identidad']]
    BscsMod['Estado'] = BscsMod['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    BscsMod['Estado'] = BscsMod['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    BscsMod['Estado'] = BscsMod['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)

    BscsMod.loc[BscsMod['Estado'] != 'Activo','Estado']='Inactivo'
    BscsMod.loc[BscsMod['Modulo'] == 'NO HAY','Estado']='Inactivo'
    BscsMod['Nombre'] = BscsMod['Nombre'].replace('\s+', ' ', regex=True)
    BscsMod['Nombre'] = BscsMod['Nombre'].replace('\t', ' ', regex=True)
    BscsMod['Nombre'] = BscsMod['Nombre'].astype(str).str.replace("|", "", regex=True)
    BscsMod['Nombre'] = BscsMod['Nombre'].astype(str).str.replace(".", "", regex=True)
    BscsMod['Nombre'] = BscsMod['Nombre'].astype(str).str.replace("á", "a", regex=True)
    BscsMod['Nombre'] = BscsMod['Nombre'].astype(str).str.replace("é", "e", regex=True)
    BscsMod['Nombre'] = BscsMod['Nombre'].astype(str).str.replace("í", "i", regex=True)
    BscsMod['Nombre'] = BscsMod['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    BscsMod['Nombre'] = BscsMod['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    BscsMod['Nombre'] = BscsMod['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    BscsMod['Nombre'] = [ele.lstrip(' ') for ele in BscsMod['Nombre']]
    BscsMod['Nombre'] = [ele.rstrip(' ') for ele in BscsMod['Nombre']]
    BscsMod = BscsMod.fillna({ 'Nombre': 'Sin nombre'})
    BscsMod.insert(loc = 0,column = 'Documento',value = '0')
    BscsMod = BscsMod.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
if not midb.empty:    
    filtro5 = BscsMod[BscsMod.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro2 = BscsMod[BscsMod.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = BscsMod[BscsMod.Nombre.isin(lista3)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = BscsMod[BscsMod.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    
    filtro6 = BscsMod[BscsMod.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = BscsMod[BscsMod.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    BscsMod = pd.concat([filtro5, filtro2])
    BscsMod = pd.concat([BscsMod, filtro3])
    BscsMod = pd.concat([BscsMod, filtro4])
    BscsMod = pd.concat([BscsMod, filtro6])
    BscsMod = pd.concat([BscsMod, filtro7])
    BscsMod.index = range(BscsMod.shape[0])
    BscsMod=BscsMod[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce','Modulo', 'DocumentoRRHH']]
    BscsMod= BscsMod.drop_duplicates(BscsMod.columns[~BscsMod.columns.isin(['Cruce'])], keep='first')
    BscsMod=BscsMod.assign(Usuario_Aplicativo='BSCS MODULOS')

    listBscsMod = BscsMod['Identidad'].to_list()
    if "sin identidad" in listBscsMod:
        listBscsMod.remove("sin identidad")
    lista5 = lista5 + listBscsMod
    BscsMod1=BscsMod[['Identidad', 'DocumentoRRHH']]
    BscsMod1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,BscsMod1])
    Nube5= Nube5.drop_duplicates( keep='first')

if BscsMod.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['BSCS MODULOS'],
    'Modulo':['N/A']}
    BscsMod = pd.DataFrame(datos)
Nube = pd.concat([Nube, BscsMod])
Nube.index = range(Nube.shape[0])

In [ ]:
Nube

MAHINDRA

In [ ]:
file_exists_Mahindra = os.path.exists(filenameT+"\\MAHINDRA.csv")

print(file_exists_Mahindra)
Mahindra = pd.DataFrame()
if file_exists_Mahindra == True:
    try:
        Mahindra=pd.read_csv(filenameT+"\\MAHINDRA.csv", sep=';',encoding = "utf-8-sig" , engine='python') #IMPORTAR
    except:
        print("Ejecutó latin -1")
        Mahindra=pd.read_csv(filenameT+"\\MAHINDRA.csv", sep=';',encoding = "latin-1" , engine='python') #IMPORTAR
    Mahindra.columns = Mahindra.columns.str.strip()
    Mahindra=Mahindra[['LOGIN_ID','USER_NAME','STATUS']] #USAR SOLO ESAS TRES COLUMNAS
    Mahindra.rename(columns={'LOGIN_ID':'Identidad', 'USER_NAME':'Nombre', 'STATUS':'Estado'}, inplace=True)
    Mahindra["Nombre"]= Mahindra["Nombre"].str.lower()
    Mahindra["Identidad"]= Mahindra["Identidad"].str.lower()
    Mahindra['Identidad'] = [ele.lstrip(' ') for ele in Mahindra['Identidad']]
    Mahindra['Identidad'] = [ele.rstrip(' ') for ele in Mahindra['Identidad']]
    Mahindra['Identidad'] = [ele.lstrip('.') for ele in Mahindra['Identidad']]
    Mahindra['Identidad'] = [ele.rstrip('.') for ele in Mahindra['Identidad']]
    Mahindra['Estado'] = Mahindra['Estado'].astype(str).str.replace("Y", "Activo", regex=True)
    Mahindra['Estado'] = Mahindra['Estado'].astype(str).str.replace("S", "Inactivo", regex=True)
    Mahindra['Estado'] = Mahindra['Estado'].astype(str).str.replace("N", "Inactivo", regex=True)
    
    Mahindra['Nombre'] = Mahindra['Nombre'].replace('\s+', ' ', regex=True)
    Mahindra['Nombre'] = Mahindra['Nombre'].replace('\t', ' ', regex=True)
    Mahindra['Nombre'] = Mahindra['Nombre'].astype(str).str.replace("|", "", regex=True)
    Mahindra['Nombre'] = Mahindra['Nombre'].astype(str).str.replace(".", "", regex=True)
    Mahindra['Nombre'] = Mahindra['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Mahindra['Nombre'] = Mahindra['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Mahindra['Nombre'] = Mahindra['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Mahindra['Nombre'] = Mahindra['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Mahindra['Nombre'] = Mahindra['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Mahindra['Nombre'] = Mahindra['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Mahindra['Nombre'] = [ele.lstrip(' ') for ele in Mahindra['Nombre']]
    Mahindra['Nombre'] = [ele.rstrip(' ') for ele in Mahindra['Nombre']]
    Mahindra.insert(loc = 0,column = 'Documento',value = '0')
    Mahindra = Mahindra.fillna({ 'Identidad': 'sin identidad'})
    

In [ ]:
Mahindra

In [ ]:
if file_exists_Mahindra == True:
    filtro5 = Mahindra[Mahindra.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro2 = Mahindra[Mahindra.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Mahindra[Mahindra.Nombre.isin(lista3)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Mahindra[Mahindra.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Mahindra[Mahindra.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Mahindra[Mahindra.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    Mahindra = pd.concat([filtro5, filtro2])
    Mahindra = pd.concat([Mahindra, filtro3])
    Mahindra = pd.concat([Mahindra, filtro4])
    Mahindra = pd.concat([Mahindra, filtro6])
    Mahindra = pd.concat([Mahindra, filtro7])
    Mahindra.index = range(Mahindra.shape[0])
    Mahindra=Mahindra[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Mahindra= Mahindra.drop_duplicates(Mahindra.columns[~Mahindra.columns.isin(['Cruce'])], keep='first')
    Mahindra=Mahindra.assign(Usuario_Aplicativo='MAHINDRA')
    Mahindra.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listMahindra = Mahindra['Identidad'].to_list()
    if "sin identidad" in listMahindra:
        listMahindra.remove("sin identidad")
    lista5 = lista5 + listMahindra
    Mahindra1=Mahindra[['Identidad', 'DocumentoRRHH']]
    Mahindra1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Mahindra1])
    Nube5= Nube5.drop_duplicates( keep='first')

if Mahindra.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['MAHINDRA'],
    'Modulo':['N/A']}
    Mahindra = pd.DataFrame(datos)
Nube = pd.concat([Nube, Mahindra])
Nube.index = range(Nube.shape[0])

In [ ]:
Nube

INTEGRADOR WEB

In [ ]:
file_exists_Integrador = os.path.exists(filenameT+"\\INTEGRADORWEB.txt")

print(file_exists_Integrador)
Integrador = pd.DataFrame()
if file_exists_Integrador == True:

    Integrador=pd.read_csv(filenameT+"\\INTEGRADORWEB.txt", sep=';',encoding = "ISO-8859-1" , engine='python') #IMPORTAR
    Integrador.columns = Integrador.columns.str.strip()
    Integrador.rename(columns={'Usuario Conectado':'Identidad', 'Nombres Apellidos':'Nombre'}, inplace=True)
    Integrador=Integrador[['Identidad','Nombre']] #USAR SOLO ESAS TRES COLUMNAS
    Integrador.insert(loc = 2,column = 'Estado',value = 'Activo')
    Integrador['Identidad'] = Integrador['Identidad'].astype(str)
    Integrador["Nombre"]= Integrador["Nombre"].str.lower()
    Integrador["Identidad"]= Integrador["Identidad"].str.lower()
    Integrador['Identidad'] = [ele.lstrip(' ') for ele in Integrador['Identidad']]
    Integrador['Identidad'] = [ele.rstrip(' ') for ele in Integrador['Identidad']]
    Integrador['Identidad'] = [ele.lstrip('.') for ele in Integrador['Identidad']]
    Integrador['Identidad'] = [ele.rstrip('.') for ele in Integrador['Identidad']]
    Integrador['Nombre'] = Integrador['Nombre'].replace('\s+', ' ', regex=True)
    Integrador['Nombre'] = Integrador['Nombre'].replace('\t', ' ', regex=True)
    Integrador['Nombre'] = Integrador['Nombre'].astype(str).str.replace("|", "", regex=True)
    Integrador['Nombre'] = Integrador['Nombre'].astype(str).str.replace(".", "", regex=True)
    Integrador['Nombre'] = Integrador['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Integrador['Nombre'] = Integrador['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Integrador['Nombre'] = Integrador['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Integrador['Nombre'] = Integrador['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Integrador['Nombre'] = Integrador['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Integrador['Nombre'] = Integrador['Nombre'].astype(str).str.replace("ò", "o", regex=True)
    Integrador['Nombre'] = Integrador['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Integrador['Nombre'] = [ele.lstrip(' ') for ele in Integrador['Nombre']]
    Integrador['Nombre'] = [ele.rstrip(' ') for ele in Integrador['Nombre']]
    Integrador.insert(loc = 0,column = 'Documento',value = '0')
    Integrador = Integrador.fillna({ 'Identidad': 'sin identidad'})
    

In [ ]:
Integrador

In [ ]:
if file_exists_Integrador == True:
    filtro5 = Integrador[Integrador.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro2 = Integrador[Integrador.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Integrador[Integrador.Nombre.isin(lista3)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Integrador[Integrador.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Integrador[Integrador.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Integrador[Integrador.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    Integrador = pd.concat([filtro5, filtro2])
    Integrador = pd.concat([Integrador, filtro3])
    Integrador = pd.concat([Integrador, filtro4])
    Integrador = pd.concat([Integrador, filtro6])
    Integrador = pd.concat([Integrador, filtro7])
    Integrador.index = range(Integrador.shape[0])
    Integrador=Integrador[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Integrador= Integrador.drop_duplicates(Integrador.columns[~Integrador.columns.isin(['Cruce'])], keep='first')
    Integrador=Integrador.assign(Usuario_Aplicativo='INTEGRADOR WEB')
    Integrador.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listIntegrador = Integrador['Identidad'].to_list()
    if "sin identidad" in listIntegrador:
        listIntegrador.remove("sin identidad")
    lista5 = lista5 + listIntegrador
    Integrador1=Integrador[['Identidad', 'DocumentoRRHH']]
    Integrador1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Integrador1])
    Nube5= Nube5.drop_duplicates( keep='first')
if Integrador.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['INTEGRADOR WEB'],
    'Modulo':['N/A']}
    Integrador = pd.DataFrame(datos)
Nube = pd.concat([Nube, Integrador])
Nube.index = range(Nube.shape[0])

In [ ]:
Nube

INTEC

In [ ]:
file_exists_Intec = os.path.exists(filenameT+"\\INTEC.csv")

print(file_exists_Intec)
Intec = pd.DataFrame()
if file_exists_Intec == True:

    Intec=pd.read_csv(filenameT+"\\INTEC.csv",sep=',',encoding = "utf-8-sig" , engine='python', ) #IMPORTAR
    Intec=Intec[['User ID','User name','User status']] #USAR SOLO ESAS TRES COLUMNAS
    Intec.rename(columns={'User ID':'Identidad','User name':'Nombre', 'User status':'Estado'}, inplace=True)
    nom = Intec["Nombre"].str.split("CSG", n=1, expand=True)
    nom= pd.DataFrame(nom)
    nom.columns = ['1', '2']
    Intec = pd.concat([Intec, nom], axis=1)
    Intec=Intec[['Identidad','1','Estado']] #USAR SOLO ESAS TRES COLUMNAS
    Intec.rename(columns={'1':'Nombre'}, inplace=True)
    Intec["Nombre"]= Intec["Nombre"].str.lower()
    Intec["Identidad"]= Intec["Identidad"].str.lower()
    Intec['Identidad'] = [ele.lstrip(' ') for ele in Intec['Identidad']]
    Intec['Identidad'] = [ele.rstrip(' ') for ele in Intec['Identidad']]
    Intec['Identidad'] = [ele.lstrip('.') for ele in Intec['Identidad']]
    Intec['Identidad'] = [ele.rstrip('.') for ele in Intec['Identidad']]
    Intec['Nombre'] = Intec['Nombre'].replace('\s+', ' ', regex=True)
    Intec['Nombre'] = Intec['Nombre'].replace('\t', ' ', regex=True)
    Intec['Nombre'] = Intec['Nombre'].astype(str).str.replace("|", "", regex=True)
    Intec['Nombre'] = Intec['Nombre'].astype(str).str.replace(".", "", regex=True)
    Intec['Nombre'] = Intec['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Intec['Nombre'] = Intec['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Intec['Nombre'] = Intec['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Intec['Nombre'] = Intec['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Intec['Nombre'] = Intec['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Intec['Nombre'] = Intec['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Intec['Nombre'] = Intec['Nombre'].astype(str).str.replace("-", "", regex=True)
    Intec['Estado'] = Intec['Estado'].astype(str).str.replace("Disabled", "Inactivo", regex=True)
    Intec['Estado'] = Intec['Estado'].astype(str).str.replace("Enabled", "Activo", regex=True)
    Intec['Nombre'] = [ele.lstrip(' ') for ele in Intec['Nombre']]
    Intec['Nombre'] = [ele.rstrip(' ') for ele in Intec['Nombre']]
    Intec.insert(loc = 0,column = 'Documento',value = '0')
    Intec = Intec.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
if file_exists_Intec == True:    
    filtro5 = Intec[Intec.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro2 = Intec[Intec.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Intec[Intec.Nombre.isin(lista3)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Intec[Intec.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Intec[Intec.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Intec[Intec.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    
    Intec = pd.concat([filtro5, filtro2])
    Intec = pd.concat([Intec, filtro3])
    Intec = pd.concat([Intec, filtro4])
    Intec = pd.concat([Intec, filtro6])
    Intec = pd.concat([Intec, filtro7])
    Intec.index = range(Intec.shape[0])
    Intec=Intec[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Intec= Intec.drop_duplicates(Intec.columns[~Intec.columns.isin(['Cruce'])], keep='first')
    Intec=Intec.assign(Usuario_Aplicativo='INTEC')
    Intec.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listIntec = Intec['Identidad'].to_list()
    if "sin identidad" in listIntec:
        listIntec.remove("sin identidad")
    lista5 = lista5 + listIntec
    Intec1=Intec[['Identidad', 'DocumentoRRHH']]
    Intec1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Intec1])
    Nube5= Nube5.drop_duplicates( keep='first')
if Intec.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['INTEC'],
    'Modulo':['N/A']}
    Intec = pd.DataFrame(datos)
Nube = pd.concat([Nube, Intec])
Nube.index = range(Nube.shape[0])

In [ ]:
Nube

MAXIMO

In [ ]:
file_exists_Maximo = os.path.exists(filenameT+"\\MAXIMO.csv")

print(file_exists_Maximo)
Maximo = pd.DataFrame()
if file_exists_Maximo == True:

    Maximo=pd.read_csv(filenameT+"\\MAXIMO.csv", sep=",", encoding='latin-1', engine='python' ) #IMPORTAR
    Maximo.columns = Maximo.columns.str.strip()
    Maximo=Maximo[['USERID','DISPLAYNAME','STATUS']] #USAR SOLO ESAS TRES COLUMNAS
    Maximo.rename(columns={'USERID':'Identidad','DISPLAYNAME':'Nombre', 'STATUS':'Estado'}, inplace=True)
    Maximo["Nombre"]= Maximo["Nombre"].str.lower()
    Maximo["Identidad"]= Maximo["Identidad"].str.lower()
    Maximo['Identidad'] = [ele.lstrip(' ') for ele in Maximo['Identidad']]
    Maximo['Identidad'] = [ele.rstrip(' ') for ele in Maximo['Identidad']]
    Maximo['Identidad'] = [ele.lstrip('.') for ele in Maximo['Identidad']]
    Maximo['Identidad'] = [ele.rstrip('.') for ele in Maximo['Identidad']]
    Maximo['Nombre'] = Maximo['Nombre'].replace('\s+', ' ', regex=True)
    Maximo['Nombre'] = Maximo['Nombre'].replace('\t', ' ', regex=True)
    Maximo['Nombre'] = Maximo['Nombre'].astype(str).str.replace("|", "", regex=True)
    Maximo['Nombre'] = Maximo['Nombre'].astype(str).str.replace(".", "", regex=True)
    Maximo['Nombre'] = Maximo['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Maximo['Nombre'] = Maximo['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Maximo['Nombre'] = Maximo['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Maximo['Nombre'] = Maximo['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Maximo['Nombre'] = Maximo['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Maximo['Nombre'] = Maximo['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Maximo['Estado'] = Maximo['Estado'].astype(str).str.replace("INACTIVE", "Inactivo", regex=True)
    Maximo['Estado'] = Maximo['Estado'].astype(str).str.replace("ACTIVE", "Activo", regex=True)
    Maximo['Nombre'] = [ele.lstrip(' ') for ele in Maximo['Nombre']]
    Maximo['Nombre'] = [ele.rstrip(' ') for ele in Maximo['Nombre']]
    Maximo.insert(loc = 0,column = 'Documento',value = '0')
    Maximo = Maximo.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
Maximo

In [ ]:
if file_exists_Maximo == True:
    filtro5 = Maximo[Maximo.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad') 
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro2 = Maximo[Maximo.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Maximo[Maximo.Nombre.isin(lista3)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Maximo[Maximo.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Maximo[Maximo.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Maximo[Maximo.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    
    Maximo = pd.concat([filtro5, filtro2])
    Maximo = pd.concat([Maximo, filtro3])
    Maximo = pd.concat([Maximo, filtro4])
    Maximo = pd.concat([Maximo, filtro6])
    Maximo = pd.concat([Maximo, filtro7])
    Maximo.index = range(Maximo.shape[0])
    Maximo=Maximo[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Maximo= Maximo.drop_duplicates(Maximo.columns[~Maximo.columns.isin(['Cruce'])], keep='first')
    Maximo=Maximo.assign(Usuario_Aplicativo='MAXIMO')
    Maximo.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listMaximo = Maximo['Identidad'].to_list()
    if "sin identidad" in listMaximo:
        listMaximo.remove("sin identidad")
    lista5 = lista5 + listMaximo
    Maximo1=Maximo[['Identidad', 'DocumentoRRHH']]
    Maximo1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Maximo1])
    Nube5= Nube5.drop_duplicates( keep='first')

if Maximo.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['MAXIMO'],
    'Modulo':['N/A']}
    Maximo = pd.DataFrame(datos)

Nube = pd.concat([Nube, Maximo])
Nube.index = range(Nube.shape[0])

In [ ]:
Nube

AZURE DEVOPS

In [ ]:
file_exists_Azure = os.path.exists(filenameT+"\\Azure.xlsx")#Pregunta si en la ruta existe el archivo

print(file_exists_Azure) #muestra false o true 
Azure = pd.DataFrame()
if file_exists_Azure == True: #Si el archivo existe

    Azure=pd.read_excel(filenameT+"\\Azure.xlsx") #importa el archivo csv separado por punto y comas
    Azure=Azure[['Username','Name']] #USAR SOLO ESAS TRES COLUMNAS
    Azure.rename(columns={'Username':'Identidad','Name':'Nombre'}, inplace=True)
    Azure["Nombre"]= Azure["Nombre"].str.lower()
    Azure["Identidad"]= Azure["Identidad"].str.lower()
    Azure['Nombre'] = Azure['Nombre'].replace('\s+', ' ', regex=True)
    Azure['Nombre'] = Azure['Nombre'].replace('\t', ' ', regex=True)
    Azure['Nombre'] = Azure['Nombre'].astype(str).str.replace("|", "", regex=True)
    Azure['Nombre'] = Azure['Nombre'].astype(str).str.replace(".", "", regex=True)
    Azure['Nombre'] = Azure['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Azure['Nombre'] = Azure['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Azure['Nombre'] = Azure['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Azure['Nombre'] = Azure['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Azure['Nombre'] = Azure['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Azure['Nombre'] = Azure['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Azure.insert(loc = 0,column = 'Documento',value = '0')
    Azure.insert(loc = 3,column = 'Estado',value = 'Activo')
    Id = Azure["Identidad"].str.split("@", n=1, expand=True)
    Id= pd.DataFrame(Id)
    Id.columns = ['Id', 'correo']

    Azure = pd.concat([Azure, Id], axis=1)
    Azure=Azure[['Id','Nombre','Estado']] #USAR SOLO ESAS TRES COLUMNAS
    Azure.rename(columns={'Id':'Identidad'}, inplace=True)
    Azure['Identidad'] = [ele.lstrip(' ') for ele in Azure['Identidad']]
    Azure['Identidad'] = [ele.rstrip(' ') for ele in Azure['Identidad']]
    Azure['Identidad'] = [ele.lstrip('.') for ele in Azure['Identidad']]
    Azure['Identidad'] = [ele.rstrip('.') for ele in Azure['Identidad']]
    Azure['Nombre'] = [ele.lstrip(' ') for ele in Azure['Nombre']]
    Azure['Nombre'] = [ele.rstrip(' ') for ele in Azure['Nombre']]
    Azure.insert(loc = 0,column = 'Documento',value = '0')
    Azure = Azure.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
if file_exists_Azure == True:
    filtro5 = Azure[Azure.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad')
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro2 = Azure[Azure.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Azure[Azure.Nombre.isin(lista3)]#filtra llave1
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Azure[Azure.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)


    filtro6 = Azure[Azure.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Azure[Azure.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    Azure = pd.concat([ filtro5, filtro2])
    Azure = pd.concat([ Azure, filtro3])
    Azure = pd.concat([ Azure, filtro4])
    Azure = pd.concat([ Azure, filtro6])
    Azure = pd.concat([ Azure, filtro7])
    Azure.index = range(Azure.shape[0])
    Azure=Azure[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH']]
    Azure= Azure.drop_duplicates(Azure.columns[~Azure.columns.isin(['Cruce'])], keep='first')
    Azure=Azure.assign(Usuario_Aplicativo='AZURE DEVOPS')
    Azure.insert(loc = 5,column = 'Modulo',value = 'N/A')

    listAzure = Azure['Identidad'].to_list()
    if "sin identidad" in listAzure:
        listAzure.remove("sin identidad")
    lista5 = lista5 + listAzure
    Azure1=Azure[['Identidad', 'DocumentoRRHH']]
    Azure1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Azure1])
    Nube5= Nube5.drop_duplicates( keep='first')
if Azure.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['AZURE DEVOPS'],
    'Modulo':['N/A']}
    Azure = pd.DataFrame(datos)
Nube = pd.concat([Nube, Azure])
Nube.index = range(Nube.shape[0])

SWITCH

<!-- switch -->

In [ ]:
file_exists_Switch = os.path.exists(filenameT+"\\SWITCH.xlsx")

print(file_exists_Switch)
Switch = pd.DataFrame()
if file_exists_Switch == True:

    resto=pd.read_excel(filenameT+"\\SWITCH.xlsx",sheet_name='RESTO') #IMPORTAR
    resto.columns = resto.columns.str.strip()
    resto=resto[['Texto antes del delimitador','Column1','Source.Name']] #USAR SOLO ESAS TRES COLUMNAS
    resto.rename(columns={'Texto antes del delimitador':'Identidad','Column1':'Nombre', 'Source.Name':'Modulo'}, inplace=True)
    resto.insert(loc = 0,column = 'Documento',value = '0')
    resto.insert(loc = 4,column = 'Estado',value = 'Activo')
    resto["Nombre"]= resto["Nombre"].str.lower()

    spg=pd.read_excel(filenameT+"\\SWITCH.xlsx",sheet_name='SPG') #IMPORTAR
    spg.columns = spg.columns.str.strip()
    spg=spg[['Portal User Name','Real Name','Source.Name', 'Status']]
    spg.rename(columns={'Portal User Name':'Identidad','Real Name':'Nombre', 'Source.Name':'Modulo', 'Status':'Estado'}, inplace=True)
    spg.insert(loc = 0,column = 'Documento',value = '0')
    spg["Nombre"]= spg["Nombre"].str.lower()


    triara=pd.read_excel(filenameT+"\\SWITCH.xlsx",sheet_name='TRIARA-SUBA') #IMPORTAR
    triara.columns = triara.columns.str.strip()
    triara=triara[['USUARIO','Source.Name', 'ESTADO']]
    triara.rename(columns={'USUARIO':'Identidad','Source.Name':'Modulo', 'ESTADO':'Estado'}, inplace=True)
    triara.insert(loc = 0,column = 'Documento',value = '0')
    triara.insert(loc = 0,column = 'Nombre',value = 'Sin Nombre')

    Switch=pd.concat([resto,spg])
    Switch=pd.concat([Switch,triara])



    Switch['Modulo'] = Switch['Modulo'].str.slice(stop=-14)

   
    Switch.rename(columns={'mod1':'Modulo'}, inplace=True)

    
    Switch["Identidad"]= Switch["Identidad"].str.lower()
    Switch['Identidad'] = [ele.lstrip(' ') for ele in Switch['Identidad']]
    Switch['Identidad'] = [ele.rstrip(' ') for ele in Switch['Identidad']]
    Switch['Identidad'] = [ele.lstrip('.') for ele in Switch['Identidad']]
    Switch['Identidad'] = [ele.rstrip('.') for ele in Switch['Identidad']]
    Switch['Nombre'] = Switch['Nombre'].replace('\s+', ' ', regex=True)
    Switch['Nombre'] = Switch['Nombre'].replace('\t', ' ', regex=True)
    Switch['Nombre'] = Switch['Nombre'].astype(str).str.replace("|", "", regex=True)
    Switch['Nombre'] = Switch['Nombre'].astype(str).str.replace(".", "", regex=True)
    Switch['Nombre'] = Switch['Nombre'].astype(str).str.replace("á", "a", regex=True)
    Switch['Nombre'] = Switch['Nombre'].astype(str).str.replace("é", "e", regex=True)
    Switch['Nombre'] = Switch['Nombre'].astype(str).str.replace("í", "i", regex=True)
    Switch['Nombre'] = Switch['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    Switch['Nombre'] = Switch['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    Switch['Nombre'] = Switch['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
    Switch=Switch.replace({"Disabled": "Inactivo", "Enabled": "Activo", "Unlocked":"Activo"}) # Cambia los diferent
    Switch['Nombre'] = [ele.lstrip(' ') for ele in Switch['Nombre']]
    Switch['Nombre'] = [ele.rstrip(' ') for ele in Switch['Nombre']]

    Switch = Switch.fillna({ 'Identidad': 'sin identidad'})

In [ ]:
if file_exists_Switch == True:
    filtro5 = Switch[Switch.Identidad.isin(lista5)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtro5=filtro5.assign(Cruce='Identidad')
    filtro5 = filtro5.merge(Nube5, how='left', left_on='Identidad', right_on='Identidad')
    filtro5.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro2 = Switch[Switch.Nombre.isin(lista2)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(Nube2, how='left', left_on='Nombre', right_on='Nombre')
    filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = Switch[Switch.Nombre.isin(lista3)]#filtra llave1
    filtro3=filtro3.assign(Cruce='Llave 3')
    filtro3 = filtro3.merge(Nube3, how='left', left_on='Nombre', right_on='Nombre')
    filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = Switch[Switch.Nombre.isin(lista4)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 4')
    filtro4 = filtro4.merge(Nube4, how='left', left_on='Nombre', right_on='Nombre')
    filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro6 = Switch[Switch.Nombre.isin(lista6)]#filtra llave3
    filtro6=filtro6.assign(Cruce='Llave 6')
    filtro6 = filtro6.merge(Nube6, how='left', left_on='Nombre', right_on='Nombre')
    filtro6.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    filtro7 = Switch[Switch.Nombre.isin(lista7)]#filtra llave4
    filtro7=filtro7.assign(Cruce='Llave 7')
    filtro7 = filtro7.merge(Nube7, how='left', left_on='Nombre', right_on='Nombre')
    filtro7.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)

    Switch = pd.concat([ filtro5, filtro2])
    Switch = pd.concat([ Switch, filtro3])
    Switch = pd.concat([ Switch, filtro4])
    Switch = pd.concat([ Switch, filtro6])
    Switch = pd.concat([ Switch, filtro7])
    Switch.index = range(Switch.shape[0])
    Switch=Switch[['Documento', 'Identidad', 'Nombre', 'Estado', 'Cruce', 'DocumentoRRHH', 'Modulo']]
    Switch= Switch.drop_duplicates(Switch.columns[~Switch.columns.isin(['Cruce'])], keep='first')
    Switch=Switch.assign(Usuario_Aplicativo='SWITCH')

    listSwitch = Switch['Identidad'].to_list()
    if "sin identidad" in listSwitch:
        listSwitch.remove("sin identidad")
    lista5 = lista5 + listSwitch
    Switch1=Switch[['Identidad', 'DocumentoRRHH']]
    Switch1.rename(columns={'DocumentoRRHH':'Documento1'}, inplace=True)
    Nube5=pd.concat([Nube5,Switch1])
    Nube5= Nube5.drop_duplicates( keep='first')
if Switch.empty:
    
    datos = {'Documento':['0'],
    'Identidad':['Sin identidad'],
    'Nombre':['Sin Nombre'],
    'Estado':['N/E'],
    'Cruce':['No cruza'],
    'DocumentoRRHH':['0'],
    'Usuario_Aplicativo':['SWITCH'],
    'Modulo':['N/A']}
    Switch = pd.DataFrame(datos)
Nube = pd.concat([Nube, Switch])
Nube.index = range(Nube.shape[0])

In [ ]:
Nube

In [ ]:
Cedulas2=Usuarios[['DOCUMENTO']] 
Cedulas2.rename(columns={'DOCUMENTO':'DocumentoRRHH'}, inplace=True)
filtro8 = Cedulas2[~Cedulas2.DocumentoRRHH.isin(Nube.DocumentoRRHH)]
filtro8.index = range(filtro8.shape[0])
filtro8=filtro8.assign(Usuario_Aplicativo='Sin Aplicativos')
Nube = pd.concat([Nube, filtro8])
# Nube.index = range(Nube.shape[0])
# Nube = Nube.fillna({'Identidad': 'Sin identidad', 'Nombre':'No encontrado','Estado':'N/E','Cruce': 'Ninguno', 'Modulo':'N/A','Usuario_Aplicativo':'Sin DA-CA-OIM'})

In [ ]:
Nube

In [ ]:
Usuarios1=Usuarios[['DOCUMENTO','NOMBRE','FECHA DE RETIRO','CASO','TIPO DE CONTRATO']]
Usuarios1= Usuarios1.drop_duplicates()
Usuarios1.rename(columns={'DOCUMENTO':'DocumentoRRHH','NOMBRE':'NombreRRHH'}, inplace=True)
Nube = Nube.merge(Usuarios1, how='left', left_on='DocumentoRRHH', right_on='DocumentoRRHH')
Nube = Nube.fillna({'Documento': '0','Identidad': 'Sin Identidad','Estado':'N/E', 'Modulo':'N/A','Cruce':'N/E','Nombre': 'Sin nombre','apellido1': '3', 'apellido2': '4'})

In [ ]:
Nube

In [ ]:
file_exists_SSFF_Activos = os.path.exists(filename+"\\Informe_Activos_Claro.xlsx")#Pregunta si en la ruta existe el archivo

print(file_exists_SSFF_Activos)#muestra false o true 

if file_exists_SSFF_Activos == True:#Si el archivo existe

    SsffA=pd.read_excel(filename+"\\Informe_Activos_Claro.xlsx")
    SsffA.columns = SsffA.columns.str.strip()
    SsffA=SsffA[['Cédula','Nombres', 'Apellidos' ,'Detalles de empleo Fecha de ingreso','Estado de colaborador']] #Usar solo las columnas que necesitamos
    SsffA.rename(columns={'Estado de colaborador':'STATUS', 'Cédula':'CUSTOM08', 'Detalles de empleo Fecha de ingreso':'Fecha Ingreso SSFF'}, inplace=True)
    SsffA.insert(loc = 5,column = 'Fecha Terminacion SSFF',value = '0')
    SsffA=SsffA[['STATUS',  'CUSTOM08', 'Fecha Ingreso SSFF', 'Fecha Terminacion SSFF','Nombres', 'Apellidos']]

    SsffA = SsffA.sort_values('Fecha Ingreso SSFF', ascending=False)
    SsffA['Fecha Ingreso SSFF'] = pd.to_datetime(SsffA['Fecha Ingreso SSFF'], format="%d/%m/%Y %H:%M:%S" , errors='coerce')
    SsffA['Fecha Ingreso SSFF'] = SsffA['Fecha Ingreso SSFF'].dt.floor('D')
    pd.options.display.float_format = '{:.0f}'.format
    SsffA['CUSTOM08']= SsffA['CUSTOM08'].astype('Int64')

    SsffA['CUSTOM08'] = SsffA['CUSTOM08'].astype(str)
    SsffA['Nombres'] = SsffA['Nombres'].astype(str)
    SsffA['Apellidos'] = SsffA['Apellidos'].astype(str)
    
    SsffA = SsffA.dropna(subset=["CUSTOM08"])
    
    SsffA['CUSTOM08'] = [ele.lstrip('0') for ele in SsffA['CUSTOM08']]
    SsffA['CUSTOM08'] = [ele.lstrip(' ') for ele in SsffA['CUSTOM08']]
    SsffA['CUSTOM08'] = [ele.rstrip(' ') for ele in SsffA['CUSTOM08']]

    SsffA['Nombres'] = [ele.lstrip(' ') for ele in SsffA['Nombres']]
    SsffA['Apellidos'] = [ele.lstrip(' ') for ele in SsffA['Apellidos']]
    SsffA['Nombres'] = [ele.rstrip(' ') for ele in SsffA['Nombres']]
    SsffA['Apellidos'] = [ele.rstrip(' ') for ele in SsffA['Apellidos']]
    
    SsffA = SsffA.dropna(subset=["Nombres"])

    SsffA=SsffA.drop_duplicates(SsffA.columns[~SsffA.columns.isin(['STATUS', 'Fecha Ingreso SSFF', 'Fecha Terminacion SSFF','Nombres', 'Apellidos'])], keep='first')
    Ssff1=SsffA[['STATUS','CUSTOM08','Nombres', 'Apellidos']]
    Ssff1.rename(columns={'STATUS':'Estado'}, inplace=True)
    SsffA=SsffA[['STATUS',  'CUSTOM08', 'Fecha Ingreso SSFF', 'Fecha Terminacion SSFF']]

    Nube = Nube.merge(SsffA, how='left', left_on='DocumentoRRHH', right_on='CUSTOM08')
    Nube = Nube.fillna({'STATUS': 'Inactivo'}) 
    
    Nube['Fecha Terminacion SSFF'] = pd.to_datetime(Nube['Fecha Terminacion SSFF'], format="%d/%m/%Y" , errors='ignore')
    Nube['FECHA DE RETIRO'] = pd.to_datetime(Nube['FECHA DE RETIRO'], format="%Y/%m/%d" , errors='coerce')
else:
    sys.exit("no existe el archivo SSFF de Activos")  

In [ ]:
Nube

In [ ]:
Nube = Nube.fillna({'Nombre': 'Sin Nombre', 'Documento': '0','Identidad': 'Sin identidad'})
Nube.loc[(Nube['Nombre'].isnull()) | (Nube['Nombre']=="") | (Nube['Nombre'].isna()), 'Nombre']='Sin Nombre'
Nube.loc[(Nube['Documento'].isnull()) | (Nube['Documento']=="") | (Nube['Documento'].isna()), 'Documento']='0'

In [ ]:
Nube['Nombre'] = Nube['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
Nube['NombreRRHH'] = Nube['NombreRRHH'].astype(str).str.replace("ñ", "n", regex=True)
Nube=Nube[['Documento','Identidad','Nombre','Estado','Cruce','Modulo','DocumentoRRHH','Usuario_Aplicativo','NombreRRHH','FECHA DE RETIRO','CASO','TIPO DE CONTRATO','STATUS', 'Fecha Ingreso SSFF', 'Fecha Terminacion SSFF']]
name = Nube["NombreRRHH"].str.split( n=4, expand=True)

try:
    name.columns = ['nombre1', 'nombre2','apellido1', 'apellido2', 'quinto']
except:
    name.columns = ['nombre1', 'nombre2','apellido1', 'apellido2']
    name.insert(loc = 4,column = 'quinto',value = '5')

name = name.fillna({'nombre1': '1', 'nombre2': '2','apellido1': '3', 'apellido2': '4', 'quinto': '5'})
Nube=pd.concat([ Nube, name], axis=1)
def match_string(NombreGrande, nombreChiquito):
    s = NombreGrande.find(nombreChiquito)
    if s == -1:
        return "No encontrado"
    else:
        return s

Nube['Nombre1_exist'] = Nube.apply(lambda x: match_string(x['Nombre'],x['nombre1']), axis=1)
Nube['Nombre2_exist'] = Nube.apply(lambda x: match_string(x['Nombre'],x['nombre2']), axis=1)
Nube['Apellido1_exist'] = Nube.apply(lambda x: match_string(x['Nombre'],x['apellido1']), axis=1)
Nube['Apellido2_exist'] = Nube.apply(lambda x: match_string(x['Nombre'],x['apellido2']), axis=1)
Nube['Quinto_exist'] = Nube.apply(lambda x: match_string(x['Nombre'],x['quinto']), axis=1)

In [ ]:
Nube.dtypes

In [ ]:
def sumarCruces(nom1,nom2,ape1,ape2, quinto):
    suma=0
    if nom1 != "No encontrado":
        suma=suma+25
    if nom2 != "No encontrado":
        suma=suma+25
    if ape1 != "No encontrado":
        suma=suma+25
    if ape2 != "No encontrado":
        suma=suma+25
    if quinto != "No encontrado":
        suma=suma+25
    return suma

Nube['coincidencia nombre APP vs SSFF'] = Nube.apply(lambda x: sumarCruces(x['Nombre1_exist'],x['Nombre2_exist'],x['Apellido1_exist'],x['Apellido2_exist'],x['Quinto_exist']), axis=1)


In [ ]:
Nube = Nube.drop(['nombre1','nombre2','apellido1',  'apellido2','Nombre1_exist','Nombre2_exist','Apellido1_exist','Apellido2_exist','quinto','Quinto_exist' ], axis=1)
Nube.rename(columns={'STATUS':'STATUS SSFF'}, inplace=True)

In [ ]:
Nube=Nube.assign(Espacios=Nube['Nombre'].str.count(' '))
Nube.insert(loc=14, column='Aplica nombre', value  = 'si')
Nube.loc[(Nube['Cruce'] == 'Identidad')& (Nube['Nombre'] != 'Sin Nombre') & (Nube['coincidencia nombre APP vs SSFF'] <50) , 'Aplica nombre']='no'
Nube.loc[(Nube['Cruce'] == 'Identidad')& (Nube['Nombre'] != 'Sin Nombre') & (Nube['coincidencia nombre APP vs SSFF'] ==0) , 'Aplica nombre']='no'
Nube.loc[(Nube['Cruce'] == 'Identidad')& (Nube['Nombre'] != 'Sin Nombre')& (Nube['coincidencia nombre APP vs SSFF'] ==25)&  (Nube['Espacios']==0), 'Aplica nombre']='si'

Nube = Nube.drop('Espacios', axis=1)
def comprobarNombre (nombre):
    nombrebb = ['csm', 'consultor','call ', 'asesor', 'cav ', 'eliminacion',  'gestor', 'ventas', 'libre']
    for c in nombrebb:
        s = nombre.find(c)
        if s != -1:
            return "si"
Nube['contiene'] = Nube.apply(lambda x: comprobarNombre(x['Nombre']), axis=1)
Nube.loc[(Nube['Cruce'] == 'Identidad')& (Nube['Nombre'] != 'Sin Nombre')& (Nube['Nombre'].notnull())& (Nube['contiene']=="si"), 'Aplica nombre']='si'
Nube = Nube.drop('contiene', axis=1)

In [ ]:
fecha_actual = datetime.datetime.now().date()
hora_actual = datetime.datetime.now().time()
hora_actual = hora_actual.strftime('%H/%M/%S')
ayer = str(fecha_actual - datetime.timedelta(days=1))

In [ ]:
Nube

In [ ]:
aplicativos = {'DA CLAROCO', 'DA COMCEL','DA COATTLA', 'CA', 'OIM', 'CRM-ONYX',
    'MOTOR DE PAGOS', 'POLIEDRO', 'RR', 'SAP 6.0', 'SGA','SICACOM',
    'AC', 'ASCARD', 'CMAX', 'SAP CLOUD','MYIT', 'CALLIDUS',
    'BSCS MODULOS', 'MAHINDRA', 'INTEGRADOR WEB', 'INTEC', 'MAXIMO', 'AZURE DEVOPS','AGENDAMIENTO', 'PENTAHO', 'SWITCH'}
for c in aplicativos: 
    sumame=Nube.apply(lambda x: x['Usuario_Aplicativo'] == c and x['Aplica nombre']=="si", axis=1).sum()
    if sumame == 0:
        datos = {'Documento':['0'],'Identidad':['Sin identidad'],'Nombre':['Sin Nombre'],'Estado':['N/E'],
        'Cruce':['No cruza'],'DocumentoRRHH':['0'],'Usuario_Aplicativo':[c],'Modulo':['N/A'],
        'Aplica nombre':['si'], 'NombreRRHH':['Sin Nombre'], 'FECHA DE RETIRO':[ayer],'CASO':['Sin caso'],
        'TIPO DE CONTRATO':['Sin contrato'],'coincidencia nombre APP vs SSFF':[0] , 'STATUS SSFF':['N/E']}
        aplicativo = pd.DataFrame(datos)
        Nube = pd.concat([Nube, aplicativo])
Nube = Nube.fillna({'NombreRRHH': 'Sin Nombre', 'CASO': 'Sin caso', 'STATUS SSFF':'N/E',
'coincidencia nombre APP vs SSFF':0})
Nube.loc[(Nube['TIPO DE CONTRATO'].isnull()) | (Nube['TIPO DE CONTRATO']=="") | (Nube['TIPO DE CONTRATO'].isna()) | (Nube['TIPO DE CONTRATO']=="nan"), 'TIPO DE CONTRATO']='Sin contrato'
Nube.loc[(Nube['NombreRRHH'].isnull()) | (Nube['NombreRRHH']=="") | (Nube['NombreRRHH'].isna()), 'NombreRRHH']='Sin Nombre'
Nube.loc[(Nube['Documento'].isnull()) | (Nube['Documento']=="") | (Nube['Documento'].isna()) | (Nube['Documento']=="nan"), 'Documento']='0'
Nube.index = range(Nube.shape[0])

In [ ]:
Nube

In [ ]:
identidadesCA= CaI[['Identidad']]
identidadesCA=identidadesCA['Identidad'].to_list()
Nube['Existe en CA'] = Nube['Identidad'].apply(lambda x: "si" if x in identidadesCA else "no")

In [ ]:
Nube=Nube.assign(Reintegro='N/A')

In [ ]:
Nube['Fuzzy Nom'] = Nube.apply(lambda x: fuzz.ratio(x['Nombre'],x['NombreRRHH']), axis=1)


In [ ]:
Nube['Documento']=Nube['Documento'].astype(str)
Nube['DocumentoRRHH']=Nube['DocumentoRRHH'].astype(str)
Nube['Fuzzy Doc'] = Nube.apply(lambda x: fuzz.ratio(x['Documento'],x['DocumentoRRHH']), axis=1)
#Nube['DocumentoRRHH']=Nube['DocumentoRRHH'].astype('Int64')
Nube["Documento"] = pd.to_numeric(Nube["Documento"], errors='ignore')
Nube['Documento']=Nube['Documento'].astype(str)

In [ ]:
Nube

In [ ]:

Nube = Nube.fillna({'Fecha Terminacion SSFF': '0'})


      
def comprobarInactivacion (app, cruce, porcentaje, Fn, Fd, nombre, documento):
      
      Espacios=nombre.count(' ')

      if cruce == "Documento" and Fd==100:
         return "si"
      elif cruce == "Documento" and Fd>=70 and(Fn>=70 or porcentaje>=50):
         return "si"
      elif Fd >= 70 and Fn >= 70:
         return "si"
      elif Fd >=70 and porcentaje>=50:
         return "si"
      elif (Fn>=70 or porcentaje>=50)and app=="ASCARD":
         return "si"
      elif nombre =="Sin Nombre" and documento=="0":
          return "si"
      elif documento=="0" and (Fn>=70 or porcentaje>=50):
          return "si"
      elif app=="Sin Aplicativos":
          return "si"
      elif Espacios==0 and porcentaje>=25 and documento=="0":
          return "si"
      else:
         return "no"
         
Nube['Aplica Inactivacion'] = Nube.apply(lambda x: comprobarInactivacion(x['Usuario_Aplicativo'],x['Cruce'],x['coincidencia nombre APP vs SSFF'],x['Fuzzy Nom'],x['Fuzzy Doc'],x['Nombre'],x['Documento'] ), axis=1)



In [ ]:
def comprobarNombre(nombre):
   nombreExcepcion = ['csm', 'consultor','call ', 'asesor', 'cav ', 'eliminacion',  'gestor', 'ventas', 'analista', 'libre', 'people','pospago', 'telem', 'representante','None', 'analista', 'supervisor', 'colombia', 'contact', 'coordinador', 'servicio', 'personalizado', 'clientes']

   for c in nombreExcepcion:
      print(c)
      s = nombre.find(c)
      if s != -1:
         return "si"
            
Nube['contiene'] = Nube.apply(lambda x: comprobarNombre(x['Nombre']), axis=1)
Nube.loc[(Nube['Documento'] == '0')& (Nube['contiene'] == 'si'), 'Aplica Inactivacion']='si'
Nube.loc[(Nube['DocumentoRRHH'] == '0'), 'Aplica Inactivacion']='no'

In [ ]:

Nube.loc[Nube['Usuario_Aplicativo']=="MYIT",'Existe en CA']='si'
fecha_actual = str(datetime.datetime.now().date())
hora_actual = datetime.datetime.now().time()
hora_actual = hora_actual.strftime('%H-%M-%S')
Nube = Nube.drop(['Aplica nombre', 'contiene'], axis=1)

In [ ]:
Nube = Nube.fillna({'FECHA DE RETIRO': ayer, 'Fecha Ingreso SSFF':'0'})

In [ ]:
Nube

In [ ]:
# Ssff1=SsffA[['STATUS','CUSTOM08','Nombres', 'Apellidos']]
Ssff1['NombreCompleto'] = Ssff1['Nombres'] + " " + Ssff1['Apellidos']
Ssff1=Ssff1[['Estado', 'CUSTOM08', 'NombreCompleto']]
Ssff1.rename(columns={'Estado': 'EstadoDocLegado'}, inplace=True)
Nube13= Nube.merge(Ssff1, how='left', left_on='Documento', right_on='CUSTOM08')
Nube13['Nombre'] = Nube13['Nombre'].astype(str)
Nube13['NombreCompleto'] = Nube13['NombreCompleto'].astype(str)
Nube13['coincidencia'] = Nube13.apply(lambda row: fuzz.token_sort_ratio(row['Nombre'], row['NombreCompleto']), axis=1)

In [ ]:
Nube13

In [ ]:
Nube13['coincidencia2'] = Nube13.apply(lambda x: fuzz.ratio(x['Nombre'], x['NombreCompleto']), axis=1)
Nube13['coincidencia3'] = Nube13.apply(lambda row: fuzz.token_set_ratio(row['Nombre'], row['NombreCompleto']), axis=1)

Nube13= Nube13.drop('CUSTOM08', axis=1)

In [ ]:
Nube13

In [ ]:
Nube13.insert(loc=22, column='Doble validacion', value='no')
Nube13.loc[(Nube13['STATUS SSFF']=="Inactivo") & (Nube13['Aplica Inactivacion']=="si") & ((Nube13['coincidencia']>=80) | (Nube13['coincidencia2']>=80) | (Nube13['coincidencia3']>=80) ) &(Nube13['Documento']!="0") , 'Doble validacion']='si'
Nube13.loc[(Nube13['STATUS SSFF']=="Inactivo") & (Nube13['Aplica Inactivacion']=="si") & ((Nube13['coincidencia']>=80) | (Nube13['coincidencia2']>=80) | (Nube13['coincidencia3']>=80) ) &(Nube13['Documento']!="0") , 'Aplica Inactivacion']='no'

In [ ]:

Nube13 = Nube13.drop(['EstadoDocLegado', 'NombreCompleto', 'coincidencia', 'coincidencia2', 'coincidencia3'], axis=1)

In [ ]:
df_filtrado = Nube13[(Nube13['Doble validacion'] == "si") & (Nube13['Identidad'] != "sin identidad")]
df_filtrado= df_filtrado[['Identidad', 'NombreRRHH']]
df_filtrado.rename(columns={'Identidad':'IdentidadValidacion', 'NombreRRHH':'NombreValidacion'}, inplace=True)

df_filtrado= df_filtrado.drop_duplicates(subset=['IdentidadValidacion'], keep='first')

Nube13=Nube13.merge(df_filtrado, how='left', left_on='Identidad', right_on='IdentidadValidacion')
Nube13['Nombre'] = Nube13['Nombre'].astype(str)
Nube13['NombreValidacion'] = Nube13['NombreValidacion'].astype(str)

Nube13['coincidencia'] = Nube13.apply(lambda row: fuzz.token_sort_ratio(row['Nombre'], row['NombreValidacion']), axis=1)
Nube13['coincidencia2'] = Nube13.apply(lambda row: fuzz.ratio(row['Nombre'], row['NombreValidacion']), axis=1)
Nube13['coincidencia3'] = Nube13.apply(lambda row: fuzz.token_set_ratio(row['Nombre'], row['NombreValidacion']), axis=1)

Nube13.loc[(Nube13['STATUS SSFF']=="Inactivo") & (Nube13['Aplica Inactivacion']=="si") & ((Nube13['coincidencia']>=80) | (Nube13['coincidencia2']>=80) | (Nube13['coincidencia3']>=80)) &((Nube13['Usuario_Aplicativo']=="AC") | (Nube13['Usuario_Aplicativo']=="ASCARD") | (Nube13['Usuario_Aplicativo']=="BSCS MODULOS") | (Nube13['Usuario_Aplicativo']=="CMAX")) , 'Doble validacion']='si'


Nube13.loc[(Nube13['STATUS SSFF']=="Inactivo") & (Nube13['Aplica Inactivacion']=="si") & ((Nube13['coincidencia']>=80) | (Nube13['coincidencia2']>=80) | (Nube13['coincidencia3']>=80)) &((Nube13['Usuario_Aplicativo']=="AC") | (Nube13['Usuario_Aplicativo']=="ASCARD") | (Nube13['Usuario_Aplicativo']=="BSCS MODULOS") | (Nube13['Usuario_Aplicativo']=="CMAX")) , 'Aplica Inactivacion']='no'

Nube13 = Nube13.drop(['IdentidadValidacion', 'NombreValidacion', 'coincidencia', 'coincidencia2', 'coincidencia3'], axis=1)

Marcar posibles homonimos

In [ ]:
Nube13.insert(loc=22, column='Posible homonimo', value='no')

Nube13.loc[(Nube13['Doble validacion']=="no") & (Nube13['Fuzzy Doc']<70) & (Nube13['Documento']!="0") & ((Nube13['Fuzzy Nom']>=70) | (Nube13['coincidencia nombre APP vs SSFF']>=50)) & (Nube13['Usuario_Aplicativo']!="ASCARD") , 'Posible homonimo']='si'

Nube13['coincidencia nombre APP vs SSFF']=Nube13['coincidencia nombre APP vs SSFF'].astype(str)+"%"

In [ ]:
nombref= fileResults + '\\IdentidadesEncontradasconSSFF '+fecha_actual+' '+hora_actual+'.csv'
Nube13.to_csv(nombref,index=False,sep=';',encoding='utf-8-sig')
nombre2= fileResults + '\\IdentidadesEncontradasconSSFF.csv'
Nube13.to_csv(nombre2,index=False,sep=';',encoding='utf-8-sig')


In [ ]:
Nube